# Import Packages and Files

In [1]:
import torch.nn as nn
from functionalities import dataloader as dl
from functionalities import evaluater as ev
from functionalities import filemanager as fm
from functionalities import trainer as tr
from functionalities import plot as p
from architecture import RotNet as RN

# Load Dataset

In [2]:
trainset, testset, classes = dl.load_cifar("./datasets")

Files already downloaded and verified
Files already downloaded and verified


In [3]:
trainloader, validloader, testloader = dl.make_dataloaders(trainset, testset, 128)

# Initialize Loss Criterion

In [4]:
criterion = nn.CrossEntropyLoss()

# Train RotNet for Rotation Task and Classifiers on Feature Maps 

In [5]:
# set rot classes
rot_classes = ['original', '90 rotation', '180 rotation', '270 rotation']

## 3 Block RotNet

In [5]:
# initialize network
net_block3 = RN.RotNet(num_classes=4, num_conv_block=3, add_avg_pool=False)

In [6]:
# train network
rot_block3_loss_log, _, rot_block3_test_accuracy_log, _, _ = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net_block3, criterion, trainloader, None, testloader, rot=['90', '180', '270'])

[1, 60] loss: 1.131
[1, 120] loss: 0.976
[1, 180] loss: 0.907
[1, 240] loss: 0.849
[1, 300] loss: 0.791
[1, 360] loss: 0.766
Epoch: 1 -> Loss: 0.680784225464
Epoch: 1 -> Test Accuracy: 69.8975
[2, 60] loss: 0.699
[2, 120] loss: 0.683
[2, 180] loss: 0.672
[2, 240] loss: 0.659
[2, 300] loss: 0.638
[2, 360] loss: 0.626
Epoch: 2 -> Loss: 0.737947106361
Epoch: 2 -> Test Accuracy: 75.53
[3, 60] loss: 0.608
[3, 120] loss: 0.595
[3, 180] loss: 0.579
[3, 240] loss: 0.557
[3, 300] loss: 0.571
[3, 360] loss: 0.547
Epoch: 3 -> Loss: 0.533422529697
Epoch: 3 -> Test Accuracy: 77.88
[4, 60] loss: 0.527
[4, 120] loss: 0.535
[4, 180] loss: 0.514
[4, 240] loss: 0.510
[4, 300] loss: 0.519
[4, 360] loss: 0.513
Epoch: 4 -> Loss: 0.522677898407
Epoch: 4 -> Test Accuracy: 80.2525
[5, 60] loss: 0.492
[5, 120] loss: 0.486
[5, 180] loss: 0.503
[5, 240] loss: 0.487
[5, 300] loss: 0.486
[5, 360] loss: 0.464
Epoch: 5 -> Loss: 0.504644989967
Epoch: 5 -> Test Accuracy: 80.67
[6, 60] loss: 0.462
[6, 120] loss: 0.472


[42, 240] loss: 0.310
[42, 300] loss: 0.320
[42, 360] loss: 0.303
Epoch: 42 -> Loss: 0.324565410614
Epoch: 42 -> Test Accuracy: 86.6675
[43, 60] loss: 0.300
[43, 120] loss: 0.299
[43, 180] loss: 0.306
[43, 240] loss: 0.317
[43, 300] loss: 0.308
[43, 360] loss: 0.325
Epoch: 43 -> Loss: 0.44145283103
Epoch: 43 -> Test Accuracy: 87.7575
[44, 60] loss: 0.302
[44, 120] loss: 0.300
[44, 180] loss: 0.304
[44, 240] loss: 0.310
[44, 300] loss: 0.315
[44, 360] loss: 0.323
Epoch: 44 -> Loss: 0.278142631054
Epoch: 44 -> Test Accuracy: 86.5475
[45, 60] loss: 0.295
[45, 120] loss: 0.288
[45, 180] loss: 0.320
[45, 240] loss: 0.310
[45, 300] loss: 0.307
[45, 360] loss: 0.303
Epoch: 45 -> Loss: 0.273495495319
Epoch: 45 -> Test Accuracy: 86.545
[46, 60] loss: 0.298
[46, 120] loss: 0.302
[46, 180] loss: 0.308
[46, 240] loss: 0.322
[46, 300] loss: 0.309
[46, 360] loss: 0.310
Epoch: 46 -> Loss: 0.282479614019
Epoch: 46 -> Test Accuracy: 86.7825
[47, 60] loss: 0.296
[47, 120] loss: 0.293
[47, 180] loss: 0.3

[83, 240] loss: 0.155
[83, 300] loss: 0.159
[83, 360] loss: 0.155
Epoch: 83 -> Loss: 0.147573813796
Epoch: 83 -> Test Accuracy: 90.2125
[84, 60] loss: 0.138
[84, 120] loss: 0.139
[84, 180] loss: 0.147
[84, 240] loss: 0.144
[84, 300] loss: 0.142
[84, 360] loss: 0.154
Epoch: 84 -> Loss: 0.0975830703974
Epoch: 84 -> Test Accuracy: 90.3
[85, 60] loss: 0.137
[85, 120] loss: 0.132
[85, 180] loss: 0.148
[85, 240] loss: 0.153
[85, 300] loss: 0.147
[85, 360] loss: 0.156
Epoch: 85 -> Loss: 0.157554775476
Epoch: 85 -> Test Accuracy: 90.2225
[86, 60] loss: 0.133
[86, 120] loss: 0.136
[86, 180] loss: 0.140
[86, 240] loss: 0.153
[86, 300] loss: 0.148
[86, 360] loss: 0.150
Epoch: 86 -> Loss: 0.118725478649
Epoch: 86 -> Test Accuracy: 90.295
[87, 60] loss: 0.144
[87, 120] loss: 0.133
[87, 180] loss: 0.135
[87, 240] loss: 0.147
[87, 300] loss: 0.150
[87, 360] loss: 0.150
Epoch: 87 -> Loss: 0.139809191227
Epoch: 87 -> Test Accuracy: 90.015
[88, 60] loss: 0.135
[88, 120] loss: 0.136
[88, 180] loss: 0.142

[123, 240] loss: 0.053
[123, 300] loss: 0.055
[123, 360] loss: 0.056
Epoch: 123 -> Loss: 0.0241162218153
Epoch: 123 -> Test Accuracy: 92.355
[124, 60] loss: 0.052
[124, 120] loss: 0.046
[124, 180] loss: 0.049
[124, 240] loss: 0.052
[124, 300] loss: 0.050
[124, 360] loss: 0.051
Epoch: 124 -> Loss: 0.0235836915672
Epoch: 124 -> Test Accuracy: 92.38
[125, 60] loss: 0.045
[125, 120] loss: 0.046
[125, 180] loss: 0.047
[125, 240] loss: 0.046
[125, 300] loss: 0.049
[125, 360] loss: 0.047
Epoch: 125 -> Loss: 0.056631308049
Epoch: 125 -> Test Accuracy: 92.3125
[126, 60] loss: 0.040
[126, 120] loss: 0.042
[126, 180] loss: 0.044
[126, 240] loss: 0.046
[126, 300] loss: 0.042
[126, 360] loss: 0.044
Epoch: 126 -> Loss: 0.0198171045631
Epoch: 126 -> Test Accuracy: 92.32
[127, 60] loss: 0.038
[127, 120] loss: 0.040
[127, 180] loss: 0.041
[127, 240] loss: 0.040
[127, 300] loss: 0.041
[127, 360] loss: 0.043
Epoch: 127 -> Loss: 0.0345460250974
Epoch: 127 -> Test Accuracy: 92.24
[128, 60] loss: 0.038
[128

[162, 360] loss: 0.014
Epoch: 162 -> Loss: 0.00652187177911
Epoch: 162 -> Test Accuracy: 92.3025
[163, 60] loss: 0.013
[163, 120] loss: 0.013
[163, 180] loss: 0.014
[163, 240] loss: 0.015
[163, 300] loss: 0.015
[163, 360] loss: 0.014
Epoch: 163 -> Loss: 0.0233575738966
Epoch: 163 -> Test Accuracy: 92.27
[164, 60] loss: 0.013
[164, 120] loss: 0.014
[164, 180] loss: 0.015
[164, 240] loss: 0.013
[164, 300] loss: 0.013
[164, 360] loss: 0.013
Epoch: 164 -> Loss: 0.00919873826206
Epoch: 164 -> Test Accuracy: 92.365
[165, 60] loss: 0.013
[165, 120] loss: 0.013
[165, 180] loss: 0.013
[165, 240] loss: 0.012
[165, 300] loss: 0.014
[165, 360] loss: 0.013
Epoch: 165 -> Loss: 0.00694509129971
Epoch: 165 -> Test Accuracy: 92.285
[166, 60] loss: 0.012
[166, 120] loss: 0.012
[166, 180] loss: 0.012
[166, 240] loss: 0.013
[166, 300] loss: 0.014
[166, 360] loss: 0.011
Epoch: 166 -> Loss: 0.022670449689
Epoch: 166 -> Test Accuracy: 92.32
[167, 60] loss: 0.012
[167, 120] loss: 0.012
[167, 180] loss: 0.013


In [7]:
# train NonLinearClassifiers on feature map of net_3block
block3_loss_log, _, block3_test_accuracy_log, _, _ = tr.train_all_blocks(3, 10, [0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 100], 0.9, 5e-4, net_block3, criterion, trainloader, None, testloader) 

[1, 60] loss: 2.171
[1, 120] loss: 1.240
[1, 180] loss: 1.114
[1, 240] loss: 1.067
[1, 300] loss: 1.028
[1, 360] loss: 0.990
Epoch: 1 -> Loss: 0.935978889465
Epoch: 1 -> Test Accuracy: 68.16
[2, 60] loss: 0.913
[2, 120] loss: 0.896
[2, 180] loss: 0.910
[2, 240] loss: 0.884
[2, 300] loss: 0.879
[2, 360] loss: 0.859
Epoch: 2 -> Loss: 0.848209738731
Epoch: 2 -> Test Accuracy: 71.56
[3, 60] loss: 0.802
[3, 120] loss: 0.815
[3, 180] loss: 0.818
[3, 240] loss: 0.781
[3, 300] loss: 0.791
[3, 360] loss: 0.771
Epoch: 3 -> Loss: 0.718413710594
Epoch: 3 -> Test Accuracy: 73.4
[4, 60] loss: 0.744
[4, 120] loss: 0.753
[4, 180] loss: 0.751
[4, 240] loss: 0.761
[4, 300] loss: 0.748
[4, 360] loss: 0.744
Epoch: 4 -> Loss: 0.582391858101
Epoch: 4 -> Test Accuracy: 74.99
[5, 60] loss: 0.729
[5, 120] loss: 0.721
[5, 180] loss: 0.717
[5, 240] loss: 0.706
[5, 300] loss: 0.729
[5, 360] loss: 0.719
Epoch: 5 -> Loss: 0.591608643532
Epoch: 5 -> Test Accuracy: 75.53
[6, 60] loss: 0.685
[6, 120] loss: 0.674
[6, 1

Epoch: 42 -> Loss: 0.499528080225
Epoch: 42 -> Test Accuracy: 82.43
[43, 60] loss: 0.316
[43, 120] loss: 0.297
[43, 180] loss: 0.300
[43, 240] loss: 0.293
[43, 300] loss: 0.321
[43, 360] loss: 0.311
Epoch: 43 -> Loss: 0.406891971827
Epoch: 43 -> Test Accuracy: 82.62
[44, 60] loss: 0.304
[44, 120] loss: 0.303
[44, 180] loss: 0.296
[44, 240] loss: 0.310
[44, 300] loss: 0.314
[44, 360] loss: 0.302
Epoch: 44 -> Loss: 0.320390552282
Epoch: 44 -> Test Accuracy: 82.83
[45, 60] loss: 0.294
[45, 120] loss: 0.283
[45, 180] loss: 0.301
[45, 240] loss: 0.303
[45, 300] loss: 0.288
[45, 360] loss: 0.295
Epoch: 45 -> Loss: 0.312586247921
Epoch: 45 -> Test Accuracy: 82.91
[46, 60] loss: 0.270
[46, 120] loss: 0.291
[46, 180] loss: 0.273
[46, 240] loss: 0.280
[46, 300] loss: 0.279
[46, 360] loss: 0.280
Epoch: 46 -> Loss: 0.194979950786
Epoch: 46 -> Test Accuracy: 82.96
[47, 60] loss: 0.265
[47, 120] loss: 0.262
[47, 180] loss: 0.281
[47, 240] loss: 0.273
[47, 300] loss: 0.284
[47, 360] loss: 0.283
Epoch

[84, 60] loss: 0.225
[84, 120] loss: 0.233
[84, 180] loss: 0.226
[84, 240] loss: 0.234
[84, 300] loss: 0.219
[84, 360] loss: 0.229
Epoch: 84 -> Loss: 0.274143129587
Epoch: 84 -> Test Accuracy: 83.05
[85, 60] loss: 0.234
[85, 120] loss: 0.235
[85, 180] loss: 0.235
[85, 240] loss: 0.224
[85, 300] loss: 0.223
[85, 360] loss: 0.221
Epoch: 85 -> Loss: 0.231608301401
Epoch: 85 -> Test Accuracy: 83.24
[86, 60] loss: 0.215
[86, 120] loss: 0.228
[86, 180] loss: 0.226
[86, 240] loss: 0.252
[86, 300] loss: 0.234
[86, 360] loss: 0.225
Epoch: 86 -> Loss: 0.314946234226
Epoch: 86 -> Test Accuracy: 83.16
[87, 60] loss: 0.224
[87, 120] loss: 0.233
[87, 180] loss: 0.233
[87, 240] loss: 0.229
[87, 300] loss: 0.230
[87, 360] loss: 0.237
Epoch: 87 -> Loss: 0.226608663797
Epoch: 87 -> Test Accuracy: 83.24
[88, 60] loss: 0.228
[88, 120] loss: 0.227
[88, 180] loss: 0.237
[88, 240] loss: 0.228
[88, 300] loss: 0.216
[88, 360] loss: 0.233
Epoch: 88 -> Loss: 0.336185872555
Epoch: 88 -> Test Accuracy: 83.25
[89, 

[25, 360] loss: 0.237
Epoch: 25 -> Loss: 0.259282678366
Epoch: 25 -> Test Accuracy: 85.81
[26, 60] loss: 0.216
[26, 120] loss: 0.217
[26, 180] loss: 0.222
[26, 240] loss: 0.221
[26, 300] loss: 0.218
[26, 360] loss: 0.219
Epoch: 26 -> Loss: 0.146848484874
Epoch: 26 -> Test Accuracy: 85.91
[27, 60] loss: 0.197
[27, 120] loss: 0.210
[27, 180] loss: 0.209
[27, 240] loss: 0.225
[27, 300] loss: 0.213
[27, 360] loss: 0.223
Epoch: 27 -> Loss: 0.19789108634
Epoch: 27 -> Test Accuracy: 85.62
[28, 60] loss: 0.200
[28, 120] loss: 0.206
[28, 180] loss: 0.221
[28, 240] loss: 0.204
[28, 300] loss: 0.211
[28, 360] loss: 0.203
Epoch: 28 -> Loss: 0.214612439275
Epoch: 28 -> Test Accuracy: 85.59
[29, 60] loss: 0.198
[29, 120] loss: 0.190
[29, 180] loss: 0.212
[29, 240] loss: 0.191
[29, 300] loss: 0.200
[29, 360] loss: 0.209
Epoch: 29 -> Loss: 0.273387014866
Epoch: 29 -> Test Accuracy: 86.15
[30, 60] loss: 0.186
[30, 120] loss: 0.189
[30, 180] loss: 0.206
[30, 240] loss: 0.196
[30, 300] loss: 0.205
[30, 3

Epoch: 66 -> Test Accuracy: 86.68
[67, 60] loss: 0.092
[67, 120] loss: 0.096
[67, 180] loss: 0.089
[67, 240] loss: 0.091
[67, 300] loss: 0.094
[67, 360] loss: 0.089
Epoch: 67 -> Loss: 0.131401851773
Epoch: 67 -> Test Accuracy: 86.69
[68, 60] loss: 0.094
[68, 120] loss: 0.092
[68, 180] loss: 0.094
[68, 240] loss: 0.093
[68, 300] loss: 0.099
[68, 360] loss: 0.089
Epoch: 68 -> Loss: 0.0799077898264
Epoch: 68 -> Test Accuracy: 86.54
[69, 60] loss: 0.098
[69, 120] loss: 0.089
[69, 180] loss: 0.092
[69, 240] loss: 0.088
[69, 300] loss: 0.096
[69, 360] loss: 0.096
Epoch: 69 -> Loss: 0.129445925355
Epoch: 69 -> Test Accuracy: 86.51
[70, 60] loss: 0.092
[70, 120] loss: 0.085
[70, 180] loss: 0.088
[70, 240] loss: 0.093
[70, 300] loss: 0.089
[70, 360] loss: 0.090
Epoch: 70 -> Loss: 0.106624186039
Epoch: 70 -> Test Accuracy: 86.7
[71, 60] loss: 0.095
[71, 120] loss: 0.094
[71, 180] loss: 0.088
[71, 240] loss: 0.098
[71, 300] loss: 0.093
[71, 360] loss: 0.096
Epoch: 71 -> Loss: 0.134414643049
Epoch

[8, 180] loss: 1.477
[8, 240] loss: 1.493
[8, 300] loss: 1.490
[8, 360] loss: 1.490
Epoch: 8 -> Loss: 1.69367945194
Epoch: 8 -> Test Accuracy: 43.32
[9, 60] loss: 1.443
[9, 120] loss: 1.489
[9, 180] loss: 1.482
[9, 240] loss: 1.479
[9, 300] loss: 1.480
[9, 360] loss: 1.467
Epoch: 9 -> Loss: 1.49393200874
Epoch: 9 -> Test Accuracy: 43.77
[10, 60] loss: 1.452
[10, 120] loss: 1.457
[10, 180] loss: 1.476
[10, 240] loss: 1.476
[10, 300] loss: 1.458
[10, 360] loss: 1.478
Epoch: 10 -> Loss: 1.44611656666
Epoch: 10 -> Test Accuracy: 43.32
[11, 60] loss: 1.457
[11, 120] loss: 1.454
[11, 180] loss: 1.461
[11, 240] loss: 1.449
[11, 300] loss: 1.469
[11, 360] loss: 1.463
Epoch: 11 -> Loss: 1.60790133476
Epoch: 11 -> Test Accuracy: 43.73
[12, 60] loss: 1.479
[12, 120] loss: 1.450
[12, 180] loss: 1.454
[12, 240] loss: 1.448
[12, 300] loss: 1.494
[12, 360] loss: 1.440
Epoch: 12 -> Loss: 1.38102173805
Epoch: 12 -> Test Accuracy: 43.83
[13, 60] loss: 1.461
[13, 120] loss: 1.453
[13, 180] loss: 1.456
[1

Epoch: 49 -> Test Accuracy: 52.74
[50, 60] loss: 1.180
[50, 120] loss: 1.179
[50, 180] loss: 1.136
[50, 240] loss: 1.157
[50, 300] loss: 1.163
[50, 360] loss: 1.161
Epoch: 50 -> Loss: 1.12166047096
Epoch: 50 -> Test Accuracy: 52.61
[51, 60] loss: 1.173
[51, 120] loss: 1.142
[51, 180] loss: 1.160
[51, 240] loss: 1.163
[51, 300] loss: 1.152
[51, 360] loss: 1.161
Epoch: 51 -> Loss: 0.956628918648
Epoch: 51 -> Test Accuracy: 52.73
[52, 60] loss: 1.178
[52, 120] loss: 1.161
[52, 180] loss: 1.156
[52, 240] loss: 1.163
[52, 300] loss: 1.158
[52, 360] loss: 1.166
Epoch: 52 -> Loss: 1.04398977757
Epoch: 52 -> Test Accuracy: 52.8
[53, 60] loss: 1.147
[53, 120] loss: 1.134
[53, 180] loss: 1.160
[53, 240] loss: 1.155
[53, 300] loss: 1.172
[53, 360] loss: 1.169
Epoch: 53 -> Loss: 1.19881725311
Epoch: 53 -> Test Accuracy: 52.62
[54, 60] loss: 1.149
[54, 120] loss: 1.159
[54, 180] loss: 1.166
[54, 240] loss: 1.143
[54, 300] loss: 1.170
[54, 360] loss: 1.156
Epoch: 54 -> Loss: 1.16287732124
Epoch: 54 

[91, 240] loss: 1.126
[91, 300] loss: 1.133
[91, 360] loss: 1.105
Epoch: 91 -> Loss: 1.17570328712
Epoch: 91 -> Test Accuracy: 53.34
[92, 60] loss: 1.121
[92, 120] loss: 1.130
[92, 180] loss: 1.132
[92, 240] loss: 1.105
[92, 300] loss: 1.131
[92, 360] loss: 1.123
Epoch: 92 -> Loss: 1.17059707642
Epoch: 92 -> Test Accuracy: 53.2
[93, 60] loss: 1.117
[93, 120] loss: 1.123
[93, 180] loss: 1.140
[93, 240] loss: 1.137
[93, 300] loss: 1.122
[93, 360] loss: 1.117
Epoch: 93 -> Loss: 0.936094105244
Epoch: 93 -> Test Accuracy: 53.68
[94, 60] loss: 1.131
[94, 120] loss: 1.119
[94, 180] loss: 1.111
[94, 240] loss: 1.132
[94, 300] loss: 1.117
[94, 360] loss: 1.131
Epoch: 94 -> Loss: 1.03828883171
Epoch: 94 -> Test Accuracy: 53.54
[95, 60] loss: 1.119
[95, 120] loss: 1.130
[95, 180] loss: 1.127
[95, 240] loss: 1.115
[95, 300] loss: 1.130
[95, 360] loss: 1.105
Epoch: 95 -> Loss: 1.13088405132
Epoch: 95 -> Test Accuracy: 53.34
[96, 60] loss: 1.110
[96, 120] loss: 1.116
[96, 180] loss: 1.125
[96, 240] 

In [8]:
# train ConvClassifiers on feature map of net_3block
conv_block3_loss_log, _, conv_block3_test_accuracy_log, _, _ = tr.train_all_blocks(3, 10, [0.1, 0.02, 0.004, 0.0008], 
    [35, 70, 85, 100], 0.9, 5e-4, net_block3, criterion, trainloader, None, testloader, use_ConvClassifier=True) 

[1, 60] loss: 1.362
[1, 120] loss: 1.037
[1, 180] loss: 0.942
[1, 240] loss: 0.872
[1, 300] loss: 0.815
[1, 360] loss: 0.807
Epoch: 1 -> Loss: 0.792883634567
Epoch: 1 -> Test Accuracy: 70.59
[2, 60] loss: 0.715
[2, 120] loss: 0.705
[2, 180] loss: 0.713
[2, 240] loss: 0.710
[2, 300] loss: 0.695
[2, 360] loss: 0.668
Epoch: 2 -> Loss: 0.751881837845
Epoch: 2 -> Test Accuracy: 73.94
[3, 60] loss: 0.621
[3, 120] loss: 0.632
[3, 180] loss: 0.633
[3, 240] loss: 0.632
[3, 300] loss: 0.616
[3, 360] loss: 0.591
Epoch: 3 -> Loss: 0.438733756542
Epoch: 3 -> Test Accuracy: 73.93
[4, 60] loss: 0.558
[4, 120] loss: 0.559
[4, 180] loss: 0.553
[4, 240] loss: 0.595
[4, 300] loss: 0.574
[4, 360] loss: 0.568
Epoch: 4 -> Loss: 0.731389284134
Epoch: 4 -> Test Accuracy: 77.82
[5, 60] loss: 0.530
[5, 120] loss: 0.539
[5, 180] loss: 0.546
[5, 240] loss: 0.534
[5, 300] loss: 0.536
[5, 360] loss: 0.552
Epoch: 5 -> Loss: 0.509520590305
Epoch: 5 -> Test Accuracy: 77.6
[6, 60] loss: 0.513
[6, 120] loss: 0.529
[6, 1

Epoch: 42 -> Loss: 0.20328810811
Epoch: 42 -> Test Accuracy: 85.43
[43, 60] loss: 0.214
[43, 120] loss: 0.208
[43, 180] loss: 0.210
[43, 240] loss: 0.226
[43, 300] loss: 0.233
[43, 360] loss: 0.228
Epoch: 43 -> Loss: 0.269247233868
Epoch: 43 -> Test Accuracy: 85.26
[44, 60] loss: 0.209
[44, 120] loss: 0.213
[44, 180] loss: 0.216
[44, 240] loss: 0.209
[44, 300] loss: 0.228
[44, 360] loss: 0.238
Epoch: 44 -> Loss: 0.296880304813
Epoch: 44 -> Test Accuracy: 85.2
[45, 60] loss: 0.195
[45, 120] loss: 0.210
[45, 180] loss: 0.213
[45, 240] loss: 0.226
[45, 300] loss: 0.223
[45, 360] loss: 0.244
Epoch: 45 -> Loss: 0.314031511545
Epoch: 45 -> Test Accuracy: 85.3
[46, 60] loss: 0.207
[46, 120] loss: 0.222
[46, 180] loss: 0.219
[46, 240] loss: 0.221
[46, 300] loss: 0.227
[46, 360] loss: 0.236
Epoch: 46 -> Loss: 0.335136651993
Epoch: 46 -> Test Accuracy: 85.11
[47, 60] loss: 0.217
[47, 120] loss: 0.209
[47, 180] loss: 0.218
[47, 240] loss: 0.218
[47, 300] loss: 0.223
[47, 360] loss: 0.228
Epoch: 4

[84, 60] loss: 0.101
[84, 120] loss: 0.095
[84, 180] loss: 0.092
[84, 240] loss: 0.095
[84, 300] loss: 0.102
[84, 360] loss: 0.094
Epoch: 84 -> Loss: 0.194360494614
Epoch: 84 -> Test Accuracy: 86.21
[85, 60] loss: 0.095
[85, 120] loss: 0.092
[85, 180] loss: 0.095
[85, 240] loss: 0.101
[85, 300] loss: 0.102
[85, 360] loss: 0.094
Epoch: 85 -> Loss: 0.10455699265
Epoch: 85 -> Test Accuracy: 86.1
[86, 60] loss: 0.087
[86, 120] loss: 0.082
[86, 180] loss: 0.087
[86, 240] loss: 0.084
[86, 300] loss: 0.078
[86, 360] loss: 0.085
Epoch: 86 -> Loss: 0.0832486376166
Epoch: 86 -> Test Accuracy: 86.68
[87, 60] loss: 0.084
[87, 120] loss: 0.081
[87, 180] loss: 0.079
[87, 240] loss: 0.086
[87, 300] loss: 0.087
[87, 360] loss: 0.079
Epoch: 87 -> Loss: 0.0660256668925
Epoch: 87 -> Test Accuracy: 86.52
[88, 60] loss: 0.081
[88, 120] loss: 0.088
[88, 180] loss: 0.081
[88, 240] loss: 0.082
[88, 300] loss: 0.081
[88, 360] loss: 0.081
Epoch: 88 -> Loss: 0.0787039250135
Epoch: 88 -> Test Accuracy: 86.42
[89,

[25, 300] loss: 0.276
[25, 360] loss: 0.287
Epoch: 25 -> Loss: 0.171951457858
Epoch: 25 -> Test Accuracy: 84.94
[26, 60] loss: 0.253
[26, 120] loss: 0.286
[26, 180] loss: 0.262
[26, 240] loss: 0.281
[26, 300] loss: 0.264
[26, 360] loss: 0.285
Epoch: 26 -> Loss: 0.319078981876
Epoch: 26 -> Test Accuracy: 85.32
[27, 60] loss: 0.246
[27, 120] loss: 0.244
[27, 180] loss: 0.268
[27, 240] loss: 0.288
[27, 300] loss: 0.270
[27, 360] loss: 0.289
Epoch: 27 -> Loss: 0.353828132153
Epoch: 27 -> Test Accuracy: 85.33
[28, 60] loss: 0.244
[28, 120] loss: 0.264
[28, 180] loss: 0.272
[28, 240] loss: 0.273
[28, 300] loss: 0.286
[28, 360] loss: 0.286
Epoch: 28 -> Loss: 0.203239396214
Epoch: 28 -> Test Accuracy: 85.75
[29, 60] loss: 0.251
[29, 120] loss: 0.271
[29, 180] loss: 0.265
[29, 240] loss: 0.263
[29, 300] loss: 0.298
[29, 360] loss: 0.276
Epoch: 29 -> Loss: 0.330868393183
Epoch: 29 -> Test Accuracy: 84.06
[30, 60] loss: 0.254
[30, 120] loss: 0.266
[30, 180] loss: 0.268
[30, 240] loss: 0.273
[30, 

Epoch: 66 -> Loss: 0.245276257396
Epoch: 66 -> Test Accuracy: 86.88
[67, 60] loss: 0.105
[67, 120] loss: 0.103
[67, 180] loss: 0.112
[67, 240] loss: 0.109
[67, 300] loss: 0.105
[67, 360] loss: 0.115
Epoch: 67 -> Loss: 0.214009493589
Epoch: 67 -> Test Accuracy: 86.66
[68, 60] loss: 0.093
[68, 120] loss: 0.088
[68, 180] loss: 0.097
[68, 240] loss: 0.115
[68, 300] loss: 0.110
[68, 360] loss: 0.108
Epoch: 68 -> Loss: 0.0804050117731
Epoch: 68 -> Test Accuracy: 86.73
[69, 60] loss: 0.106
[69, 120] loss: 0.107
[69, 180] loss: 0.107
[69, 240] loss: 0.117
[69, 300] loss: 0.105
[69, 360] loss: 0.111
Epoch: 69 -> Loss: 0.0827852338552
Epoch: 69 -> Test Accuracy: 87.3
[70, 60] loss: 0.093
[70, 120] loss: 0.106
[70, 180] loss: 0.101
[70, 240] loss: 0.099
[70, 300] loss: 0.107
[70, 360] loss: 0.113
Epoch: 70 -> Loss: 0.159878671169
Epoch: 70 -> Test Accuracy: 87.26
[71, 60] loss: 0.084
[71, 120] loss: 0.071
[71, 180] loss: 0.067
[71, 240] loss: 0.066
[71, 300] loss: 0.066
[71, 360] loss: 0.064
Epoc

[8, 60] loss: 1.283
[8, 120] loss: 1.277
[8, 180] loss: 1.286
[8, 240] loss: 1.311
[8, 300] loss: 1.265
[8, 360] loss: 1.293
Epoch: 8 -> Loss: 1.29113197327
Epoch: 8 -> Test Accuracy: 49.39
[9, 60] loss: 1.301
[9, 120] loss: 1.279
[9, 180] loss: 1.260
[9, 240] loss: 1.284
[9, 300] loss: 1.272
[9, 360] loss: 1.277
Epoch: 9 -> Loss: 1.35623145103
Epoch: 9 -> Test Accuracy: 48.4
[10, 60] loss: 1.275
[10, 120] loss: 1.255
[10, 180] loss: 1.262
[10, 240] loss: 1.274
[10, 300] loss: 1.249
[10, 360] loss: 1.266
Epoch: 10 -> Loss: 1.29763555527
Epoch: 10 -> Test Accuracy: 50.93
[11, 60] loss: 1.277
[11, 120] loss: 1.257
[11, 180] loss: 1.260
[11, 240] loss: 1.255
[11, 300] loss: 1.266
[11, 360] loss: 1.283
Epoch: 11 -> Loss: 1.51121926308
Epoch: 11 -> Test Accuracy: 48.44
[12, 60] loss: 1.255
[12, 120] loss: 1.246
[12, 180] loss: 1.252
[12, 240] loss: 1.257
[12, 300] loss: 1.259
[12, 360] loss: 1.254
Epoch: 12 -> Loss: 1.26686346531
Epoch: 12 -> Test Accuracy: 49.95
[13, 60] loss: 1.265
[13, 1

[49, 360] loss: 1.058
Epoch: 49 -> Loss: 1.04007863998
Epoch: 49 -> Test Accuracy: 56.56
[50, 60] loss: 1.042
[50, 120] loss: 1.072
[50, 180] loss: 1.046
[50, 240] loss: 1.046
[50, 300] loss: 1.089
[50, 360] loss: 1.053
Epoch: 50 -> Loss: 1.31148076057
Epoch: 50 -> Test Accuracy: 55.81
[51, 60] loss: 1.082
[51, 120] loss: 1.051
[51, 180] loss: 1.063
[51, 240] loss: 1.061
[51, 300] loss: 1.053
[51, 360] loss: 1.060
Epoch: 51 -> Loss: 0.992772936821
Epoch: 51 -> Test Accuracy: 56.11
[52, 60] loss: 1.054
[52, 120] loss: 1.049
[52, 180] loss: 1.075
[52, 240] loss: 1.040
[52, 300] loss: 1.067
[52, 360] loss: 1.057
Epoch: 52 -> Loss: 1.18972527981
Epoch: 52 -> Test Accuracy: 55.19
[53, 60] loss: 1.036
[53, 120] loss: 1.058
[53, 180] loss: 1.054
[53, 240] loss: 1.089
[53, 300] loss: 1.045
[53, 360] loss: 1.063
Epoch: 53 -> Loss: 1.13427042961
Epoch: 53 -> Test Accuracy: 55.42
[54, 60] loss: 1.041
[54, 120] loss: 1.060
[54, 180] loss: 1.057
[54, 240] loss: 1.060
[54, 300] loss: 1.061
[54, 360]

[91, 60] loss: 0.892
[91, 120] loss: 0.867
[91, 180] loss: 0.866
[91, 240] loss: 0.885
[91, 300] loss: 0.864
[91, 360] loss: 0.891
Epoch: 91 -> Loss: 0.731607079506
Epoch: 91 -> Test Accuracy: 62.01
[92, 60] loss: 0.869
[92, 120] loss: 0.875
[92, 180] loss: 0.882
[92, 240] loss: 0.865
[92, 300] loss: 0.869
[92, 360] loss: 0.874
Epoch: 92 -> Loss: 0.920016944408
Epoch: 92 -> Test Accuracy: 61.89
[93, 60] loss: 0.863
[93, 120] loss: 0.878
[93, 180] loss: 0.884
[93, 240] loss: 0.881
[93, 300] loss: 0.871
[93, 360] loss: 0.876
Epoch: 93 -> Loss: 1.03743028641
Epoch: 93 -> Test Accuracy: 62.12
[94, 60] loss: 0.879
[94, 120] loss: 0.884
[94, 180] loss: 0.892
[94, 240] loss: 0.865
[94, 300] loss: 0.893
[94, 360] loss: 0.869
Epoch: 94 -> Loss: 0.85540086031
Epoch: 94 -> Test Accuracy: 61.94
[95, 60] loss: 0.877
[95, 120] loss: 0.887
[95, 180] loss: 0.859
[95, 240] loss: 0.872
[95, 300] loss: 0.869
[95, 360] loss: 0.880
Epoch: 95 -> Loss: 1.02554631233
Epoch: 95 -> Test Accuracy: 62.5
[96, 60] 

In [ ]:
# save variables
fm.save_variable([rot_block3_loss_log, rot_block3_test_accuracy_log, 
                  block3_loss_log, block3_test_accuracy_log, 
                  conv_block3_loss_log, conv_block3_test_accuracy_log], "3_block_net")

In [9]:
# rename files
fm.add_block_to_name(3, [100, 200])

## 4 Block RotNet

In [10]:
# initialize network
net_block4 = RN.RotNet(num_classes=4, num_conv_block=4, add_avg_pool=False)

In [11]:
# train network
rot_block4_loss_log, _, rot_block4_test_accuracy_log, _, _ = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net_block4, criterion, trainloader, None, testloader, rot=['90', '180', '270'])

[1, 60] loss: 1.150
[1, 120] loss: 1.001
[1, 180] loss: 0.922
[1, 240] loss: 0.876
[1, 300] loss: 0.833
[1, 360] loss: 0.786
Epoch: 1 -> Loss: 0.781775653362
Epoch: 1 -> Test Accuracy: 68.8775
[2, 60] loss: 0.729
[2, 120] loss: 0.725
[2, 180] loss: 0.698
[2, 240] loss: 0.684
[2, 300] loss: 0.641
[2, 360] loss: 0.656
Epoch: 2 -> Loss: 0.584232747555
Epoch: 2 -> Test Accuracy: 75.1475
[3, 60] loss: 0.621
[3, 120] loss: 0.594
[3, 180] loss: 0.595
[3, 240] loss: 0.587
[3, 300] loss: 0.584
[3, 360] loss: 0.564
Epoch: 3 -> Loss: 0.645894467831
Epoch: 3 -> Test Accuracy: 77.05
[4, 60] loss: 0.544
[4, 120] loss: 0.551
[4, 180] loss: 0.521
[4, 240] loss: 0.532
[4, 300] loss: 0.520
[4, 360] loss: 0.524
Epoch: 4 -> Loss: 0.476690769196
Epoch: 4 -> Test Accuracy: 79.5825
[5, 60] loss: 0.497
[5, 120] loss: 0.492
[5, 180] loss: 0.496
[5, 240] loss: 0.500
[5, 300] loss: 0.484
[5, 360] loss: 0.487
Epoch: 5 -> Loss: 0.413071870804
Epoch: 5 -> Test Accuracy: 80.7575
[6, 60] loss: 0.473
[6, 120] loss: 0.

[42, 240] loss: 0.304
[42, 300] loss: 0.308
[42, 360] loss: 0.322
Epoch: 42 -> Loss: 0.24705453217
Epoch: 42 -> Test Accuracy: 87.725
[43, 60] loss: 0.308
[43, 120] loss: 0.302
[43, 180] loss: 0.311
[43, 240] loss: 0.315
[43, 300] loss: 0.301
[43, 360] loss: 0.313
Epoch: 43 -> Loss: 0.284123122692
Epoch: 43 -> Test Accuracy: 86.16
[44, 60] loss: 0.311
[44, 120] loss: 0.296
[44, 180] loss: 0.311
[44, 240] loss: 0.321
[44, 300] loss: 0.301
[44, 360] loss: 0.305
Epoch: 44 -> Loss: 0.261650413275
Epoch: 44 -> Test Accuracy: 87.1175
[45, 60] loss: 0.300
[45, 120] loss: 0.308
[45, 180] loss: 0.308
[45, 240] loss: 0.308
[45, 300] loss: 0.299
[45, 360] loss: 0.309
Epoch: 45 -> Loss: 0.355645358562
Epoch: 45 -> Test Accuracy: 87.105
[46, 60] loss: 0.304
[46, 120] loss: 0.297
[46, 180] loss: 0.302
[46, 240] loss: 0.293
[46, 300] loss: 0.307
[46, 360] loss: 0.314
Epoch: 46 -> Loss: 0.321740746498
Epoch: 46 -> Test Accuracy: 86.2225
[47, 60] loss: 0.296
[47, 120] loss: 0.305
[47, 180] loss: 0.299


[83, 240] loss: 0.148
[83, 300] loss: 0.142
[83, 360] loss: 0.164
Epoch: 83 -> Loss: 0.129991531372
Epoch: 83 -> Test Accuracy: 90.685
[84, 60] loss: 0.135
[84, 120] loss: 0.132
[84, 180] loss: 0.148
[84, 240] loss: 0.144
[84, 300] loss: 0.152
[84, 360] loss: 0.151
Epoch: 84 -> Loss: 0.122033432126
Epoch: 84 -> Test Accuracy: 90.7275
[85, 60] loss: 0.140
[85, 120] loss: 0.135
[85, 180] loss: 0.140
[85, 240] loss: 0.143
[85, 300] loss: 0.144
[85, 360] loss: 0.158
Epoch: 85 -> Loss: 0.0600674524903
Epoch: 85 -> Test Accuracy: 90.425
[86, 60] loss: 0.124
[86, 120] loss: 0.133
[86, 180] loss: 0.136
[86, 240] loss: 0.149
[86, 300] loss: 0.143
[86, 360] loss: 0.155
Epoch: 86 -> Loss: 0.141129478812
Epoch: 86 -> Test Accuracy: 90.59
[87, 60] loss: 0.132
[87, 120] loss: 0.139
[87, 180] loss: 0.147
[87, 240] loss: 0.140
[87, 300] loss: 0.147
[87, 360] loss: 0.144
Epoch: 87 -> Loss: 0.127103477716
Epoch: 87 -> Test Accuracy: 90.5525
[88, 60] loss: 0.139
[88, 120] loss: 0.138
[88, 180] loss: 0.14

[123, 240] loss: 0.047
[123, 300] loss: 0.048
[123, 360] loss: 0.046
Epoch: 123 -> Loss: 0.0228638090193
Epoch: 123 -> Test Accuracy: 92.5825
[124, 60] loss: 0.044
[124, 120] loss: 0.045
[124, 180] loss: 0.042
[124, 240] loss: 0.047
[124, 300] loss: 0.043
[124, 360] loss: 0.046
Epoch: 124 -> Loss: 0.032440982759
Epoch: 124 -> Test Accuracy: 92.4675
[125, 60] loss: 0.039
[125, 120] loss: 0.041
[125, 180] loss: 0.039
[125, 240] loss: 0.039
[125, 300] loss: 0.038
[125, 360] loss: 0.041
Epoch: 125 -> Loss: 0.0157813932747
Epoch: 125 -> Test Accuracy: 92.535
[126, 60] loss: 0.037
[126, 120] loss: 0.033
[126, 180] loss: 0.039
[126, 240] loss: 0.039
[126, 300] loss: 0.040
[126, 360] loss: 0.037
Epoch: 126 -> Loss: 0.0508316271007
Epoch: 126 -> Test Accuracy: 92.5525
[127, 60] loss: 0.032
[127, 120] loss: 0.035
[127, 180] loss: 0.038
[127, 240] loss: 0.036
[127, 300] loss: 0.037
[127, 360] loss: 0.036
Epoch: 127 -> Loss: 0.0440130233765
Epoch: 127 -> Test Accuracy: 92.5325
[128, 60] loss: 0.03

[162, 360] loss: 0.013
Epoch: 162 -> Loss: 0.0078805796802
Epoch: 162 -> Test Accuracy: 92.2875
[163, 60] loss: 0.012
[163, 120] loss: 0.010
[163, 180] loss: 0.011
[163, 240] loss: 0.011
[163, 300] loss: 0.011
[163, 360] loss: 0.011
Epoch: 163 -> Loss: 0.00940859504044
Epoch: 163 -> Test Accuracy: 92.4025
[164, 60] loss: 0.010
[164, 120] loss: 0.010
[164, 180] loss: 0.011
[164, 240] loss: 0.009
[164, 300] loss: 0.010
[164, 360] loss: 0.010
Epoch: 164 -> Loss: 0.0069149537012
Epoch: 164 -> Test Accuracy: 92.4375
[165, 60] loss: 0.009
[165, 120] loss: 0.009
[165, 180] loss: 0.011
[165, 240] loss: 0.008
[165, 300] loss: 0.010
[165, 360] loss: 0.010
Epoch: 165 -> Loss: 0.0208732225001
Epoch: 165 -> Test Accuracy: 92.4625
[166, 60] loss: 0.009
[166, 120] loss: 0.009
[166, 180] loss: 0.009
[166, 240] loss: 0.010
[166, 300] loss: 0.008
[166, 360] loss: 0.011
Epoch: 166 -> Loss: 0.0108460579067
Epoch: 166 -> Test Accuracy: 92.3875
[167, 60] loss: 0.009
[167, 120] loss: 0.008
[167, 180] loss: 0

In [12]:
# train NonLinearClassifiers on feature map of net_3block
block4_loss_log, _, block4_test_accuracy_log, _, _ = tr.train_all_blocks(4, 10, [0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 100], 0.9, 5e-4, net_block4, criterion, trainloader, None, testloader) 

[1, 60] loss: 2.238
[1, 120] loss: 1.261
[1, 180] loss: 1.156
[1, 240] loss: 1.097
[1, 300] loss: 1.037
[1, 360] loss: 1.002
Epoch: 1 -> Loss: 0.864320397377
Epoch: 1 -> Test Accuracy: 67.59
[2, 60] loss: 0.958
[2, 120] loss: 0.947
[2, 180] loss: 0.895
[2, 240] loss: 0.903
[2, 300] loss: 0.886
[2, 360] loss: 0.873
Epoch: 2 -> Loss: 0.746971905231
Epoch: 2 -> Test Accuracy: 71.14
[3, 60] loss: 0.795
[3, 120] loss: 0.840
[3, 180] loss: 0.824
[3, 240] loss: 0.805
[3, 300] loss: 0.822
[3, 360] loss: 0.830
Epoch: 3 -> Loss: 0.729731142521
Epoch: 3 -> Test Accuracy: 73.89
[4, 60] loss: 0.808
[4, 120] loss: 0.759
[4, 180] loss: 0.779
[4, 240] loss: 0.755
[4, 300] loss: 0.752
[4, 360] loss: 0.766
Epoch: 4 -> Loss: 0.678737938404
Epoch: 4 -> Test Accuracy: 74.01
[5, 60] loss: 0.738
[5, 120] loss: 0.744
[5, 180] loss: 0.750
[5, 240] loss: 0.730
[5, 300] loss: 0.733
[5, 360] loss: 0.737
Epoch: 5 -> Loss: 0.692242085934
Epoch: 5 -> Test Accuracy: 75.33
[6, 60] loss: 0.703
[6, 120] loss: 0.700
[6, 

Epoch: 42 -> Loss: 0.354045569897
Epoch: 42 -> Test Accuracy: 82.32
[43, 60] loss: 0.328
[43, 120] loss: 0.307
[43, 180] loss: 0.335
[43, 240] loss: 0.308
[43, 300] loss: 0.315
[43, 360] loss: 0.313
Epoch: 43 -> Loss: 0.410318374634
Epoch: 43 -> Test Accuracy: 82.41
[44, 60] loss: 0.316
[44, 120] loss: 0.294
[44, 180] loss: 0.314
[44, 240] loss: 0.309
[44, 300] loss: 0.322
[44, 360] loss: 0.311
Epoch: 44 -> Loss: 0.43537157774
Epoch: 44 -> Test Accuracy: 82.66
[45, 60] loss: 0.301
[45, 120] loss: 0.311
[45, 180] loss: 0.304
[45, 240] loss: 0.309
[45, 300] loss: 0.317
[45, 360] loss: 0.299
Epoch: 45 -> Loss: 0.335314333439
Epoch: 45 -> Test Accuracy: 82.78
[46, 60] loss: 0.300
[46, 120] loss: 0.294
[46, 180] loss: 0.297
[46, 240] loss: 0.294
[46, 300] loss: 0.294
[46, 360] loss: 0.292
Epoch: 46 -> Loss: 0.316843062639
Epoch: 46 -> Test Accuracy: 83.09
[47, 60] loss: 0.284
[47, 120] loss: 0.296
[47, 180] loss: 0.289
[47, 240] loss: 0.292
[47, 300] loss: 0.292
[47, 360] loss: 0.300
Epoch:

[84, 60] loss: 0.257
[84, 120] loss: 0.246
[84, 180] loss: 0.244
[84, 240] loss: 0.238
[84, 300] loss: 0.248
[84, 360] loss: 0.252
Epoch: 84 -> Loss: 0.350840747356
Epoch: 84 -> Test Accuracy: 82.97
[85, 60] loss: 0.259
[85, 120] loss: 0.244
[85, 180] loss: 0.235
[85, 240] loss: 0.243
[85, 300] loss: 0.244
[85, 360] loss: 0.253
Epoch: 85 -> Loss: 0.240564629436
Epoch: 85 -> Test Accuracy: 82.98
[86, 60] loss: 0.255
[86, 120] loss: 0.249
[86, 180] loss: 0.254
[86, 240] loss: 0.256
[86, 300] loss: 0.245
[86, 360] loss: 0.245
Epoch: 86 -> Loss: 0.224809333682
Epoch: 86 -> Test Accuracy: 82.97
[87, 60] loss: 0.242
[87, 120] loss: 0.225
[87, 180] loss: 0.247
[87, 240] loss: 0.246
[87, 300] loss: 0.260
[87, 360] loss: 0.246
Epoch: 87 -> Loss: 0.407147020102
Epoch: 87 -> Test Accuracy: 83.05
[88, 60] loss: 0.246
[88, 120] loss: 0.239
[88, 180] loss: 0.246
[88, 240] loss: 0.232
[88, 300] loss: 0.257
[88, 360] loss: 0.254
Epoch: 88 -> Loss: 0.194575741887
Epoch: 88 -> Test Accuracy: 83.04
[89, 

[25, 360] loss: 0.233
Epoch: 25 -> Loss: 0.300466686487
Epoch: 25 -> Test Accuracy: 85.95
[26, 60] loss: 0.223
[26, 120] loss: 0.206
[26, 180] loss: 0.207
[26, 240] loss: 0.230
[26, 300] loss: 0.232
[26, 360] loss: 0.231
Epoch: 26 -> Loss: 0.195571139455
Epoch: 26 -> Test Accuracy: 86.42
[27, 60] loss: 0.212
[27, 120] loss: 0.218
[27, 180] loss: 0.221
[27, 240] loss: 0.198
[27, 300] loss: 0.216
[27, 360] loss: 0.221
Epoch: 27 -> Loss: 0.248861312866
Epoch: 27 -> Test Accuracy: 86.08
[28, 60] loss: 0.200
[28, 120] loss: 0.200
[28, 180] loss: 0.208
[28, 240] loss: 0.205
[28, 300] loss: 0.221
[28, 360] loss: 0.231
Epoch: 28 -> Loss: 0.247321411967
Epoch: 28 -> Test Accuracy: 86.31
[29, 60] loss: 0.203
[29, 120] loss: 0.205
[29, 180] loss: 0.196
[29, 240] loss: 0.217
[29, 300] loss: 0.203
[29, 360] loss: 0.204
Epoch: 29 -> Loss: 0.273793399334
Epoch: 29 -> Test Accuracy: 86.1
[30, 60] loss: 0.202
[30, 120] loss: 0.196
[30, 180] loss: 0.216
[30, 240] loss: 0.199
[30, 300] loss: 0.210
[30, 3

Epoch: 66 -> Test Accuracy: 86.99
[67, 60] loss: 0.100
[67, 120] loss: 0.095
[67, 180] loss: 0.091
[67, 240] loss: 0.096
[67, 300] loss: 0.099
[67, 360] loss: 0.095
Epoch: 67 -> Loss: 0.107475541532
Epoch: 67 -> Test Accuracy: 87.05
[68, 60] loss: 0.096
[68, 120] loss: 0.104
[68, 180] loss: 0.098
[68, 240] loss: 0.089
[68, 300] loss: 0.095
[68, 360] loss: 0.098
Epoch: 68 -> Loss: 0.0491379313171
Epoch: 68 -> Test Accuracy: 87.13
[69, 60] loss: 0.098
[69, 120] loss: 0.097
[69, 180] loss: 0.097
[69, 240] loss: 0.092
[69, 300] loss: 0.093
[69, 360] loss: 0.102
Epoch: 69 -> Loss: 0.0430906303227
Epoch: 69 -> Test Accuracy: 86.96
[70, 60] loss: 0.092
[70, 120] loss: 0.093
[70, 180] loss: 0.095
[70, 240] loss: 0.095
[70, 300] loss: 0.088
[70, 360] loss: 0.095
Epoch: 70 -> Loss: 0.110732994974
Epoch: 70 -> Test Accuracy: 87.04
[71, 60] loss: 0.093
[71, 120] loss: 0.091
[71, 180] loss: 0.090
[71, 240] loss: 0.088
[71, 300] loss: 0.089
[71, 360] loss: 0.097
Epoch: 71 -> Loss: 0.159320011735
Epo

[8, 120] loss: 0.524
[8, 180] loss: 0.527
[8, 240] loss: 0.534
[8, 300] loss: 0.525
[8, 360] loss: 0.542
Epoch: 8 -> Loss: 0.47324219346
Epoch: 8 -> Test Accuracy: 78.48
[9, 60] loss: 0.523
[9, 120] loss: 0.518
[9, 180] loss: 0.526
[9, 240] loss: 0.535
[9, 300] loss: 0.528
[9, 360] loss: 0.543
Epoch: 9 -> Loss: 0.716715335846
Epoch: 9 -> Test Accuracy: 78.55
[10, 60] loss: 0.518
[10, 120] loss: 0.518
[10, 180] loss: 0.528
[10, 240] loss: 0.522
[10, 300] loss: 0.525
[10, 360] loss: 0.522
Epoch: 10 -> Loss: 0.647689819336
Epoch: 10 -> Test Accuracy: 78.1
[11, 60] loss: 0.518
[11, 120] loss: 0.498
[11, 180] loss: 0.511
[11, 240] loss: 0.521
[11, 300] loss: 0.519
[11, 360] loss: 0.536
Epoch: 11 -> Loss: 0.603722453117
Epoch: 11 -> Test Accuracy: 78.08
[12, 60] loss: 0.509
[12, 120] loss: 0.521
[12, 180] loss: 0.490
[12, 240] loss: 0.513
[12, 300] loss: 0.522
[12, 360] loss: 0.522
Epoch: 12 -> Loss: 0.797437250614
Epoch: 12 -> Test Accuracy: 79.33
[13, 60] loss: 0.499
[13, 120] loss: 0.505


[49, 300] loss: 0.246
[49, 360] loss: 0.246
Epoch: 49 -> Loss: 0.498691171408
Epoch: 49 -> Test Accuracy: 81.74
[50, 60] loss: 0.242
[50, 120] loss: 0.233
[50, 180] loss: 0.238
[50, 240] loss: 0.256
[50, 300] loss: 0.239
[50, 360] loss: 0.237
Epoch: 50 -> Loss: 0.240729287267
Epoch: 50 -> Test Accuracy: 81.75
[51, 60] loss: 0.229
[51, 120] loss: 0.255
[51, 180] loss: 0.236
[51, 240] loss: 0.239
[51, 300] loss: 0.249
[51, 360] loss: 0.246
Epoch: 51 -> Loss: 0.222125843167
Epoch: 51 -> Test Accuracy: 81.65
[52, 60] loss: 0.236
[52, 120] loss: 0.243
[52, 180] loss: 0.240
[52, 240] loss: 0.245
[52, 300] loss: 0.254
[52, 360] loss: 0.249
Epoch: 52 -> Loss: 0.196796581149
Epoch: 52 -> Test Accuracy: 81.89
[53, 60] loss: 0.228
[53, 120] loss: 0.238
[53, 180] loss: 0.258
[53, 240] loss: 0.233
[53, 300] loss: 0.234
[53, 360] loss: 0.235
Epoch: 53 -> Loss: 0.359286755323
Epoch: 53 -> Test Accuracy: 81.79
[54, 60] loss: 0.250
[54, 120] loss: 0.236
[54, 180] loss: 0.240
[54, 240] loss: 0.253
[54, 

Epoch: 90 -> Loss: 0.144330009818
Epoch: 90 -> Test Accuracy: 81.82
[91, 60] loss: 0.194
[91, 120] loss: 0.201
[91, 180] loss: 0.208
[91, 240] loss: 0.195
[91, 300] loss: 0.202
[91, 360] loss: 0.200
Epoch: 91 -> Loss: 0.301908165216
Epoch: 91 -> Test Accuracy: 81.86
[92, 60] loss: 0.207
[92, 120] loss: 0.204
[92, 180] loss: 0.196
[92, 240] loss: 0.213
[92, 300] loss: 0.203
[92, 360] loss: 0.203
Epoch: 92 -> Loss: 0.244701221585
Epoch: 92 -> Test Accuracy: 81.84
[93, 60] loss: 0.207
[93, 120] loss: 0.194
[93, 180] loss: 0.196
[93, 240] loss: 0.193
[93, 300] loss: 0.198
[93, 360] loss: 0.202
Epoch: 93 -> Loss: 0.266982376575
Epoch: 93 -> Test Accuracy: 81.71
[94, 60] loss: 0.190
[94, 120] loss: 0.199
[94, 180] loss: 0.210
[94, 240] loss: 0.197
[94, 300] loss: 0.211
[94, 360] loss: 0.200
Epoch: 94 -> Loss: 0.319970875978
Epoch: 94 -> Test Accuracy: 81.74
[95, 60] loss: 0.193
[95, 120] loss: 0.192
[95, 180] loss: 0.190
[95, 240] loss: 0.201
[95, 300] loss: 0.199
[95, 360] loss: 0.208
Epoch

[32, 240] loss: 1.520
[32, 300] loss: 1.506
[32, 360] loss: 1.502
Epoch: 32 -> Loss: 1.53307819366
Epoch: 32 -> Test Accuracy: 42.22
[33, 60] loss: 1.515
[33, 120] loss: 1.508
[33, 180] loss: 1.515
[33, 240] loss: 1.510
[33, 300] loss: 1.508
[33, 360] loss: 1.502
Epoch: 33 -> Loss: 1.44775021076
Epoch: 33 -> Test Accuracy: 42.15
[34, 60] loss: 1.496
[34, 120] loss: 1.500
[34, 180] loss: 1.521
[34, 240] loss: 1.503
[34, 300] loss: 1.510
[34, 360] loss: 1.515
Epoch: 34 -> Loss: 1.60223293304
Epoch: 34 -> Test Accuracy: 42.08
[35, 60] loss: 1.506
[35, 120] loss: 1.505
[35, 180] loss: 1.512
[35, 240] loss: 1.512
[35, 300] loss: 1.524
[35, 360] loss: 1.496
Epoch: 35 -> Loss: 1.43342030048
Epoch: 35 -> Test Accuracy: 41.82
[36, 60] loss: 1.526
[36, 120] loss: 1.499
[36, 180] loss: 1.506
[36, 240] loss: 1.496
[36, 300] loss: 1.515
[36, 360] loss: 1.517
Epoch: 36 -> Loss: 1.35133290291
Epoch: 36 -> Test Accuracy: 41.7
[37, 60] loss: 1.502
[37, 120] loss: 1.508
[37, 180] loss: 1.498
[37, 240] l

Epoch: 73 -> Test Accuracy: 45.46
[74, 60] loss: 1.361
[74, 120] loss: 1.369
[74, 180] loss: 1.387
[74, 240] loss: 1.389
[74, 300] loss: 1.368
[74, 360] loss: 1.396
Epoch: 74 -> Loss: 1.49426150322
Epoch: 74 -> Test Accuracy: 45.52
[75, 60] loss: 1.368
[75, 120] loss: 1.376
[75, 180] loss: 1.402
[75, 240] loss: 1.377
[75, 300] loss: 1.360
[75, 360] loss: 1.375
Epoch: 75 -> Loss: 1.41872096062
Epoch: 75 -> Test Accuracy: 45.42
[76, 60] loss: 1.366
[76, 120] loss: 1.365
[76, 180] loss: 1.396
[76, 240] loss: 1.369
[76, 300] loss: 1.405
[76, 360] loss: 1.373
Epoch: 76 -> Loss: 1.43177592754
Epoch: 76 -> Test Accuracy: 45.23
[77, 60] loss: 1.392
[77, 120] loss: 1.369
[77, 180] loss: 1.388
[77, 240] loss: 1.388
[77, 300] loss: 1.371
[77, 360] loss: 1.376
Epoch: 77 -> Loss: 1.51616621017
Epoch: 77 -> Test Accuracy: 45.09
[78, 60] loss: 1.349
[78, 120] loss: 1.394
[78, 180] loss: 1.362
[78, 240] loss: 1.388
[78, 300] loss: 1.389
[78, 360] loss: 1.381
Epoch: 78 -> Loss: 1.46561729908
Epoch: 78 

In [13]:
# train ConvClassifiers on feature map of net_3block
conv_block4_loss_log, _, conv_block4_test_accuracy_log, _, _ = tr.train_all_blocks(4, 10, [0.1, 0.02, 0.004, 0.0008], 
    [35, 70, 85, 100], 0.9, 5e-4, net_block4, criterion, trainloader, None, testloader, use_ConvClassifier=True) 

[1, 60] loss: 1.366
[1, 120] loss: 1.033
[1, 180] loss: 0.964
[1, 240] loss: 0.875
[1, 300] loss: 0.869
[1, 360] loss: 0.829
Epoch: 1 -> Loss: 0.770958185196
Epoch: 1 -> Test Accuracy: 69.65
[2, 60] loss: 0.766
[2, 120] loss: 0.757
[2, 180] loss: 0.720
[2, 240] loss: 0.713
[2, 300] loss: 0.695
[2, 360] loss: 0.684
Epoch: 2 -> Loss: 0.731605708599
Epoch: 2 -> Test Accuracy: 73.18
[3, 60] loss: 0.655
[3, 120] loss: 0.654
[3, 180] loss: 0.642
[3, 240] loss: 0.651
[3, 300] loss: 0.637
[3, 360] loss: 0.638
Epoch: 3 -> Loss: 0.696549057961
Epoch: 3 -> Test Accuracy: 76.14
[4, 60] loss: 0.592
[4, 120] loss: 0.583
[4, 180] loss: 0.593
[4, 240] loss: 0.599
[4, 300] loss: 0.595
[4, 360] loss: 0.566
Epoch: 4 -> Loss: 0.517301440239
Epoch: 4 -> Test Accuracy: 75.34
[5, 60] loss: 0.552
[5, 120] loss: 0.543
[5, 180] loss: 0.565
[5, 240] loss: 0.569
[5, 300] loss: 0.581
[5, 360] loss: 0.544
Epoch: 5 -> Loss: 0.716678977013
Epoch: 5 -> Test Accuracy: 78.01
[6, 60] loss: 0.510
[6, 120] loss: 0.545
[6, 

Epoch: 42 -> Loss: 0.268489658833
Epoch: 42 -> Test Accuracy: 85.36
[43, 60] loss: 0.216
[43, 120] loss: 0.234
[43, 180] loss: 0.236
[43, 240] loss: 0.241
[43, 300] loss: 0.226
[43, 360] loss: 0.233
Epoch: 43 -> Loss: 0.270304620266
Epoch: 43 -> Test Accuracy: 84.06
[44, 60] loss: 0.222
[44, 120] loss: 0.221
[44, 180] loss: 0.242
[44, 240] loss: 0.235
[44, 300] loss: 0.239
[44, 360] loss: 0.232
Epoch: 44 -> Loss: 0.221127554774
Epoch: 44 -> Test Accuracy: 84.92
[45, 60] loss: 0.221
[45, 120] loss: 0.218
[45, 180] loss: 0.222
[45, 240] loss: 0.231
[45, 300] loss: 0.244
[45, 360] loss: 0.244
Epoch: 45 -> Loss: 0.241934448481
Epoch: 45 -> Test Accuracy: 85.02
[46, 60] loss: 0.222
[46, 120] loss: 0.224
[46, 180] loss: 0.224
[46, 240] loss: 0.242
[46, 300] loss: 0.227
[46, 360] loss: 0.236
Epoch: 46 -> Loss: 0.179915875196
Epoch: 46 -> Test Accuracy: 85.26
[47, 60] loss: 0.215
[47, 120] loss: 0.232
[47, 180] loss: 0.233
[47, 240] loss: 0.229
[47, 300] loss: 0.223
[47, 360] loss: 0.240
Epoch

[84, 60] loss: 0.097
[84, 120] loss: 0.104
[84, 180] loss: 0.102
[84, 240] loss: 0.103
[84, 300] loss: 0.103
[84, 360] loss: 0.112
Epoch: 84 -> Loss: 0.122205749154
Epoch: 84 -> Test Accuracy: 86.15
[85, 60] loss: 0.102
[85, 120] loss: 0.101
[85, 180] loss: 0.105
[85, 240] loss: 0.101
[85, 300] loss: 0.104
[85, 360] loss: 0.103
Epoch: 85 -> Loss: 0.143395036459
Epoch: 85 -> Test Accuracy: 86.7
[86, 60] loss: 0.101
[86, 120] loss: 0.096
[86, 180] loss: 0.088
[86, 240] loss: 0.086
[86, 300] loss: 0.087
[86, 360] loss: 0.096
Epoch: 86 -> Loss: 0.119076631963
Epoch: 86 -> Test Accuracy: 86.76
[87, 60] loss: 0.084
[87, 120] loss: 0.087
[87, 180] loss: 0.084
[87, 240] loss: 0.087
[87, 300] loss: 0.090
[87, 360] loss: 0.090
Epoch: 87 -> Loss: 0.0653090029955
Epoch: 87 -> Test Accuracy: 86.87
[88, 60] loss: 0.081
[88, 120] loss: 0.090
[88, 180] loss: 0.084
[88, 240] loss: 0.088
[88, 300] loss: 0.086
[88, 360] loss: 0.099
Epoch: 88 -> Loss: 0.0823090821505
Epoch: 88 -> Test Accuracy: 86.93
[89,

[25, 300] loss: 0.277
[25, 360] loss: 0.279
Epoch: 25 -> Loss: 0.154956161976
Epoch: 25 -> Test Accuracy: 85.93
[26, 60] loss: 0.236
[26, 120] loss: 0.252
[26, 180] loss: 0.263
[26, 240] loss: 0.279
[26, 300] loss: 0.275
[26, 360] loss: 0.271
Epoch: 26 -> Loss: 0.398384422064
Epoch: 26 -> Test Accuracy: 85.32
[27, 60] loss: 0.250
[27, 120] loss: 0.248
[27, 180] loss: 0.267
[27, 240] loss: 0.275
[27, 300] loss: 0.287
[27, 360] loss: 0.280
Epoch: 27 -> Loss: 0.305144429207
Epoch: 27 -> Test Accuracy: 85.33
[28, 60] loss: 0.243
[28, 120] loss: 0.249
[28, 180] loss: 0.263
[28, 240] loss: 0.281
[28, 300] loss: 0.270
[28, 360] loss: 0.274
Epoch: 28 -> Loss: 0.189732342958
Epoch: 28 -> Test Accuracy: 84.76
[29, 60] loss: 0.246
[29, 120] loss: 0.253
[29, 180] loss: 0.267
[29, 240] loss: 0.254
[29, 300] loss: 0.278
[29, 360] loss: 0.285
Epoch: 29 -> Loss: 0.327595174313
Epoch: 29 -> Test Accuracy: 85.22
[30, 60] loss: 0.256
[30, 120] loss: 0.253
[30, 180] loss: 0.261
[30, 240] loss: 0.277
[30, 

Epoch: 66 -> Loss: 0.138274505734
Epoch: 66 -> Test Accuracy: 87.46
[67, 60] loss: 0.091
[67, 120] loss: 0.096
[67, 180] loss: 0.098
[67, 240] loss: 0.103
[67, 300] loss: 0.111
[67, 360] loss: 0.114
Epoch: 67 -> Loss: 0.11690954864
Epoch: 67 -> Test Accuracy: 86.92
[68, 60] loss: 0.092
[68, 120] loss: 0.101
[68, 180] loss: 0.108
[68, 240] loss: 0.113
[68, 300] loss: 0.107
[68, 360] loss: 0.108
Epoch: 68 -> Loss: 0.109473228455
Epoch: 68 -> Test Accuracy: 86.63
[69, 60] loss: 0.097
[69, 120] loss: 0.093
[69, 180] loss: 0.095
[69, 240] loss: 0.102
[69, 300] loss: 0.114
[69, 360] loss: 0.105
Epoch: 69 -> Loss: 0.0828288123012
Epoch: 69 -> Test Accuracy: 87.23
[70, 60] loss: 0.094
[70, 120] loss: 0.095
[70, 180] loss: 0.103
[70, 240] loss: 0.095
[70, 300] loss: 0.109
[70, 360] loss: 0.106
Epoch: 70 -> Loss: 0.12513551116
Epoch: 70 -> Test Accuracy: 87.42
[71, 60] loss: 0.073
[71, 120] loss: 0.072
[71, 180] loss: 0.063
[71, 240] loss: 0.063
[71, 300] loss: 0.055
[71, 360] loss: 0.061
Epoch:

[8, 60] loss: 0.449
[8, 120] loss: 0.446
[8, 180] loss: 0.448
[8, 240] loss: 0.452
[8, 300] loss: 0.464
[8, 360] loss: 0.459
Epoch: 8 -> Loss: 0.343017667532
Epoch: 8 -> Test Accuracy: 79.88
[9, 60] loss: 0.421
[9, 120] loss: 0.433
[9, 180] loss: 0.461
[9, 240] loss: 0.454
[9, 300] loss: 0.450
[9, 360] loss: 0.461
Epoch: 9 -> Loss: 0.394865095615
Epoch: 9 -> Test Accuracy: 80.02
[10, 60] loss: 0.428
[10, 120] loss: 0.441
[10, 180] loss: 0.452
[10, 240] loss: 0.449
[10, 300] loss: 0.444
[10, 360] loss: 0.434
Epoch: 10 -> Loss: 0.570749163628
Epoch: 10 -> Test Accuracy: 80.07
[11, 60] loss: 0.414
[11, 120] loss: 0.437
[11, 180] loss: 0.440
[11, 240] loss: 0.454
[11, 300] loss: 0.430
[11, 360] loss: 0.444
Epoch: 11 -> Loss: 0.420474380255
Epoch: 11 -> Test Accuracy: 79.96
[12, 60] loss: 0.426
[12, 120] loss: 0.445
[12, 180] loss: 0.430
[12, 240] loss: 0.457
[12, 300] loss: 0.429
[12, 360] loss: 0.433
Epoch: 12 -> Loss: 0.437109082937
Epoch: 12 -> Test Accuracy: 79.66
[13, 60] loss: 0.410


[49, 240] loss: 0.271
[49, 300] loss: 0.288
[49, 360] loss: 0.292
Epoch: 49 -> Loss: 0.36791652441
Epoch: 49 -> Test Accuracy: 83.56
[50, 60] loss: 0.261
[50, 120] loss: 0.266
[50, 180] loss: 0.251
[50, 240] loss: 0.281
[50, 300] loss: 0.267
[50, 360] loss: 0.279
Epoch: 50 -> Loss: 0.366372019053
Epoch: 50 -> Test Accuracy: 82.4
[51, 60] loss: 0.261
[51, 120] loss: 0.259
[51, 180] loss: 0.271
[51, 240] loss: 0.284
[51, 300] loss: 0.282
[51, 360] loss: 0.283
Epoch: 51 -> Loss: 0.37327042222
Epoch: 51 -> Test Accuracy: 82.4
[52, 60] loss: 0.247
[52, 120] loss: 0.271
[52, 180] loss: 0.268
[52, 240] loss: 0.263
[52, 300] loss: 0.268
[52, 360] loss: 0.292
Epoch: 52 -> Loss: 0.266908824444
Epoch: 52 -> Test Accuracy: 83.23
[53, 60] loss: 0.261
[53, 120] loss: 0.270
[53, 180] loss: 0.262
[53, 240] loss: 0.277
[53, 300] loss: 0.277
[53, 360] loss: 0.275
Epoch: 53 -> Loss: 0.290431320667
Epoch: 53 -> Test Accuracy: 82.63
[54, 60] loss: 0.253
[54, 120] loss: 0.271
[54, 180] loss: 0.261
[54, 240]

[90, 360] loss: 0.131
Epoch: 90 -> Loss: 0.148670464754
Epoch: 90 -> Test Accuracy: 84.64
[91, 60] loss: 0.125
[91, 120] loss: 0.124
[91, 180] loss: 0.127
[91, 240] loss: 0.127
[91, 300] loss: 0.129
[91, 360] loss: 0.134
Epoch: 91 -> Loss: 0.229154825211
Epoch: 91 -> Test Accuracy: 84.54
[92, 60] loss: 0.133
[92, 120] loss: 0.126
[92, 180] loss: 0.131
[92, 240] loss: 0.125
[92, 300] loss: 0.130
[92, 360] loss: 0.122
Epoch: 92 -> Loss: 0.132621482015
Epoch: 92 -> Test Accuracy: 84.62
[93, 60] loss: 0.131
[93, 120] loss: 0.122
[93, 180] loss: 0.124
[93, 240] loss: 0.126
[93, 300] loss: 0.131
[93, 360] loss: 0.122
Epoch: 93 -> Loss: 0.124566137791
Epoch: 93 -> Test Accuracy: 84.54
[94, 60] loss: 0.116
[94, 120] loss: 0.123
[94, 180] loss: 0.120
[94, 240] loss: 0.125
[94, 300] loss: 0.126
[94, 360] loss: 0.125
Epoch: 94 -> Loss: 0.141932845116
Epoch: 94 -> Test Accuracy: 84.5
[95, 60] loss: 0.120
[95, 120] loss: 0.124
[95, 180] loss: 0.128
[95, 240] loss: 0.125
[95, 300] loss: 0.119
[95, 3

[32, 180] loss: 1.480
[32, 240] loss: 1.479
[32, 300] loss: 1.470
[32, 360] loss: 1.467
Epoch: 32 -> Loss: 1.3478000164
Epoch: 32 -> Test Accuracy: 43.13
[33, 60] loss: 1.440
[33, 120] loss: 1.489
[33, 180] loss: 1.464
[33, 240] loss: 1.482
[33, 300] loss: 1.483
[33, 360] loss: 1.474
Epoch: 33 -> Loss: 1.63054466248
Epoch: 33 -> Test Accuracy: 42.0
[34, 60] loss: 1.479
[34, 120] loss: 1.476
[34, 180] loss: 1.466
[34, 240] loss: 1.478
[34, 300] loss: 1.470
[34, 360] loss: 1.480
Epoch: 34 -> Loss: 1.49120390415
Epoch: 34 -> Test Accuracy: 42.52
[35, 60] loss: 1.463
[35, 120] loss: 1.474
[35, 180] loss: 1.472
[35, 240] loss: 1.459
[35, 300] loss: 1.474
[35, 360] loss: 1.460
Epoch: 35 -> Loss: 1.38645970821
Epoch: 35 -> Test Accuracy: 43.84
[36, 60] loss: 1.431
[36, 120] loss: 1.350
[36, 180] loss: 1.358
[36, 240] loss: 1.355
[36, 300] loss: 1.347
[36, 360] loss: 1.360
Epoch: 36 -> Loss: 1.42236995697
Epoch: 36 -> Test Accuracy: 47.24
[37, 60] loss: 1.358
[37, 120] loss: 1.341
[37, 180] lo

Epoch: 73 -> Loss: 1.06943678856
Epoch: 73 -> Test Accuracy: 50.63
[74, 60] loss: 1.241
[74, 120] loss: 1.238
[74, 180] loss: 1.209
[74, 240] loss: 1.211
[74, 300] loss: 1.223
[74, 360] loss: 1.230
Epoch: 74 -> Loss: 1.23911035061
Epoch: 74 -> Test Accuracy: 50.69
[75, 60] loss: 1.214
[75, 120] loss: 1.231
[75, 180] loss: 1.221
[75, 240] loss: 1.207
[75, 300] loss: 1.211
[75, 360] loss: 1.213
Epoch: 75 -> Loss: 1.00642776489
Epoch: 75 -> Test Accuracy: 51.05
[76, 60] loss: 1.240
[76, 120] loss: 1.221
[76, 180] loss: 1.196
[76, 240] loss: 1.214
[76, 300] loss: 1.222
[76, 360] loss: 1.227
Epoch: 76 -> Loss: 1.14471018314
Epoch: 76 -> Test Accuracy: 51.04
[77, 60] loss: 1.221
[77, 120] loss: 1.230
[77, 180] loss: 1.205
[77, 240] loss: 1.224
[77, 300] loss: 1.219
[77, 360] loss: 1.219
Epoch: 77 -> Loss: 0.967162132263
Epoch: 77 -> Test Accuracy: 50.96
[78, 60] loss: 1.198
[78, 120] loss: 1.195
[78, 180] loss: 1.219
[78, 240] loss: 1.232
[78, 300] loss: 1.197
[78, 360] loss: 1.216
Epoch: 78

In [ ]:
# save variables
fm.save_variable([rot_block4_loss_log, rot_block4_test_accuracy_log, 
                  block4_loss_log, block4_test_accuracy_log, 
                  conv_block4_loss_log, conv_block4_test_accuracy_log], "4_block_net")

In [14]:
# rename files
fm.add_block_to_name(4, [100, 200])

## 5 Block RotNet

In [15]:
# initialize network
net_block5 = RN.RotNet(num_classes=4, num_conv_block=5, add_avg_pool=False)

In [16]:
# train network
rot_block5_loss_log, _, rot_block5_test_accuracy_log, _, _ = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net_block5, criterion, trainloader, None, testloader, rot=['90', '180', '270'])

[1, 60] loss: 1.150
[1, 120] loss: 1.009
[1, 180] loss: 0.937
[1, 240] loss: 0.881
[1, 300] loss: 0.840
[1, 360] loss: 0.809
Epoch: 1 -> Loss: 0.706528186798
Epoch: 1 -> Test Accuracy: 67.8
[2, 60] loss: 0.750
[2, 120] loss: 0.738
[2, 180] loss: 0.719
[2, 240] loss: 0.699
[2, 300] loss: 0.701
[2, 360] loss: 0.661
Epoch: 2 -> Loss: 0.674540877342
Epoch: 2 -> Test Accuracy: 74.25
[3, 60] loss: 0.648
[3, 120] loss: 0.628
[3, 180] loss: 0.622
[3, 240] loss: 0.613
[3, 300] loss: 0.601
[3, 360] loss: 0.599
Epoch: 3 -> Loss: 0.547346711159
Epoch: 3 -> Test Accuracy: 76.78
[4, 60] loss: 0.570
[4, 120] loss: 0.575
[4, 180] loss: 0.556
[4, 240] loss: 0.559
[4, 300] loss: 0.540
[4, 360] loss: 0.556
Epoch: 4 -> Loss: 0.583145558834
Epoch: 4 -> Test Accuracy: 78.97
[5, 60] loss: 0.542
[5, 120] loss: 0.530
[5, 180] loss: 0.518
[5, 240] loss: 0.520
[5, 300] loss: 0.500
[5, 360] loss: 0.493
Epoch: 5 -> Loss: 0.451287358999
Epoch: 5 -> Test Accuracy: 79.9525
[6, 60] loss: 0.485
[6, 120] loss: 0.493
[6,

[42, 300] loss: 0.326
[42, 360] loss: 0.311
Epoch: 42 -> Loss: 0.202413871884
Epoch: 42 -> Test Accuracy: 86.4575
[43, 60] loss: 0.303
[43, 120] loss: 0.312
[43, 180] loss: 0.308
[43, 240] loss: 0.320
[43, 300] loss: 0.327
[43, 360] loss: 0.316
Epoch: 43 -> Loss: 0.38969796896
Epoch: 43 -> Test Accuracy: 86.6225
[44, 60] loss: 0.299
[44, 120] loss: 0.314
[44, 180] loss: 0.314
[44, 240] loss: 0.318
[44, 300] loss: 0.305
[44, 360] loss: 0.313
Epoch: 44 -> Loss: 0.24058906734
Epoch: 44 -> Test Accuracy: 86.245
[45, 60] loss: 0.301
[45, 120] loss: 0.309
[45, 180] loss: 0.306
[45, 240] loss: 0.313
[45, 300] loss: 0.322
[45, 360] loss: 0.315
Epoch: 45 -> Loss: 0.296419978142
Epoch: 45 -> Test Accuracy: 86.9975
[46, 60] loss: 0.295
[46, 120] loss: 0.309
[46, 180] loss: 0.327
[46, 240] loss: 0.329
[46, 300] loss: 0.302
[46, 360] loss: 0.310
Epoch: 46 -> Loss: 0.329180210829
Epoch: 46 -> Test Accuracy: 86.6975
[47, 60] loss: 0.301
[47, 120] loss: 0.314
[47, 180] loss: 0.310
[47, 240] loss: 0.30

[83, 240] loss: 0.155
[83, 300] loss: 0.159
[83, 360] loss: 0.149
Epoch: 83 -> Loss: 0.133886381984
Epoch: 83 -> Test Accuracy: 90.1575
[84, 60] loss: 0.129
[84, 120] loss: 0.146
[84, 180] loss: 0.148
[84, 240] loss: 0.151
[84, 300] loss: 0.156
[84, 360] loss: 0.160
Epoch: 84 -> Loss: 0.134064465761
Epoch: 84 -> Test Accuracy: 89.715
[85, 60] loss: 0.141
[85, 120] loss: 0.146
[85, 180] loss: 0.140
[85, 240] loss: 0.145
[85, 300] loss: 0.151
[85, 360] loss: 0.156
Epoch: 85 -> Loss: 0.112060904503
Epoch: 85 -> Test Accuracy: 89.6125
[86, 60] loss: 0.133
[86, 120] loss: 0.148
[86, 180] loss: 0.149
[86, 240] loss: 0.147
[86, 300] loss: 0.150
[86, 360] loss: 0.153
Epoch: 86 -> Loss: 0.161302492023
Epoch: 86 -> Test Accuracy: 90.4925
[87, 60] loss: 0.142
[87, 120] loss: 0.143
[87, 180] loss: 0.141
[87, 240] loss: 0.153
[87, 300] loss: 0.149
[87, 360] loss: 0.148
Epoch: 87 -> Loss: 0.139245599508
Epoch: 87 -> Test Accuracy: 89.745
[88, 60] loss: 0.138
[88, 120] loss: 0.139
[88, 180] loss: 0.1

[123, 240] loss: 0.048
[123, 300] loss: 0.052
[123, 360] loss: 0.051
Epoch: 123 -> Loss: 0.0242857132107
Epoch: 123 -> Test Accuracy: 91.915
[124, 60] loss: 0.046
[124, 120] loss: 0.040
[124, 180] loss: 0.042
[124, 240] loss: 0.044
[124, 300] loss: 0.046
[124, 360] loss: 0.049
Epoch: 124 -> Loss: 0.0347153320909
Epoch: 124 -> Test Accuracy: 92.225
[125, 60] loss: 0.040
[125, 120] loss: 0.038
[125, 180] loss: 0.040
[125, 240] loss: 0.042
[125, 300] loss: 0.042
[125, 360] loss: 0.042
Epoch: 125 -> Loss: 0.0650164335966
Epoch: 125 -> Test Accuracy: 91.995
[126, 60] loss: 0.040
[126, 120] loss: 0.039
[126, 180] loss: 0.040
[126, 240] loss: 0.037
[126, 300] loss: 0.041
[126, 360] loss: 0.041
Epoch: 126 -> Loss: 0.0732892155647
Epoch: 126 -> Test Accuracy: 91.9675
[127, 60] loss: 0.038
[127, 120] loss: 0.039
[127, 180] loss: 0.039
[127, 240] loss: 0.036
[127, 300] loss: 0.037
[127, 360] loss: 0.037
Epoch: 127 -> Loss: 0.016385499388
Epoch: 127 -> Test Accuracy: 91.865
[128, 60] loss: 0.032
[

[162, 360] loss: 0.013
Epoch: 162 -> Loss: 0.00761998165399
Epoch: 162 -> Test Accuracy: 91.89
[163, 60] loss: 0.013
[163, 120] loss: 0.011
[163, 180] loss: 0.011
[163, 240] loss: 0.011
[163, 300] loss: 0.011
[163, 360] loss: 0.011
Epoch: 163 -> Loss: 0.0119529506192
Epoch: 163 -> Test Accuracy: 91.945
[164, 60] loss: 0.010
[164, 120] loss: 0.009
[164, 180] loss: 0.011
[164, 240] loss: 0.009
[164, 300] loss: 0.011
[164, 360] loss: 0.011
Epoch: 164 -> Loss: 0.00904276128858
Epoch: 164 -> Test Accuracy: 91.955
[165, 60] loss: 0.010
[165, 120] loss: 0.009
[165, 180] loss: 0.009
[165, 240] loss: 0.010
[165, 300] loss: 0.009
[165, 360] loss: 0.010
Epoch: 165 -> Loss: 0.0153510095552
Epoch: 165 -> Test Accuracy: 91.985
[166, 60] loss: 0.009
[166, 120] loss: 0.010
[166, 180] loss: 0.009
[166, 240] loss: 0.009
[166, 300] loss: 0.008
[166, 360] loss: 0.009
Epoch: 166 -> Loss: 0.00571954529732
Epoch: 166 -> Test Accuracy: 91.94
[167, 60] loss: 0.009
[167, 120] loss: 0.007
[167, 180] loss: 0.009


In [17]:
# train NonLinearClassifiers on feature map of net_3block
block5_loss_log, _, block5_test_accuracy_log, _, _ = tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], 
    [20, 40, 45, 100], 0.9, 5e-4, net_block5, criterion, trainloader, None, testloader) 

[1, 60] loss: 2.182
[1, 120] loss: 1.240
[1, 180] loss: 1.122
[1, 240] loss: 1.080
[1, 300] loss: 1.037
[1, 360] loss: 0.998
Epoch: 1 -> Loss: 0.888221740723
Epoch: 1 -> Test Accuracy: 67.78
[2, 60] loss: 0.941
[2, 120] loss: 0.915
[2, 180] loss: 0.901
[2, 240] loss: 0.874
[2, 300] loss: 0.886
[2, 360] loss: 0.867
Epoch: 2 -> Loss: 0.951764404774
Epoch: 2 -> Test Accuracy: 71.28
[3, 60] loss: 0.853
[3, 120] loss: 0.810
[3, 180] loss: 0.791
[3, 240] loss: 0.808
[3, 300] loss: 0.787
[3, 360] loss: 0.766
Epoch: 3 -> Loss: 0.725242853165
Epoch: 3 -> Test Accuracy: 72.86
[4, 60] loss: 0.763
[4, 120] loss: 0.781
[4, 180] loss: 0.775
[4, 240] loss: 0.751
[4, 300] loss: 0.736
[4, 360] loss: 0.771
Epoch: 4 -> Loss: 0.7917958498
Epoch: 4 -> Test Accuracy: 74.22
[5, 60] loss: 0.721
[5, 120] loss: 0.716
[5, 180] loss: 0.733
[5, 240] loss: 0.741
[5, 300] loss: 0.711
[5, 360] loss: 0.730
Epoch: 5 -> Loss: 0.757658779621
Epoch: 5 -> Test Accuracy: 75.18
[6, 60] loss: 0.704
[6, 120] loss: 0.698
[6, 18

Epoch: 42 -> Loss: 0.420903027058
Epoch: 42 -> Test Accuracy: 82.26
[43, 60] loss: 0.333
[43, 120] loss: 0.322
[43, 180] loss: 0.316
[43, 240] loss: 0.327
[43, 300] loss: 0.317
[43, 360] loss: 0.324
Epoch: 43 -> Loss: 0.316190809011
Epoch: 43 -> Test Accuracy: 82.23
[44, 60] loss: 0.308
[44, 120] loss: 0.309
[44, 180] loss: 0.305
[44, 240] loss: 0.312
[44, 300] loss: 0.319
[44, 360] loss: 0.322
Epoch: 44 -> Loss: 0.290711343288
Epoch: 44 -> Test Accuracy: 82.25
[45, 60] loss: 0.288
[45, 120] loss: 0.321
[45, 180] loss: 0.316
[45, 240] loss: 0.304
[45, 300] loss: 0.299
[45, 360] loss: 0.307
Epoch: 45 -> Loss: 0.32226151228
Epoch: 45 -> Test Accuracy: 82.22
[46, 60] loss: 0.305
[46, 120] loss: 0.306
[46, 180] loss: 0.308
[46, 240] loss: 0.286
[46, 300] loss: 0.291
[46, 360] loss: 0.297
Epoch: 46 -> Loss: 0.279633402824
Epoch: 46 -> Test Accuracy: 82.46
[47, 60] loss: 0.291
[47, 120] loss: 0.294
[47, 180] loss: 0.299
[47, 240] loss: 0.291
[47, 300] loss: 0.274
[47, 360] loss: 0.303
Epoch:

[84, 60] loss: 0.243
[84, 120] loss: 0.253
[84, 180] loss: 0.248
[84, 240] loss: 0.245
[84, 300] loss: 0.247
[84, 360] loss: 0.250
Epoch: 84 -> Loss: 0.153328850865
Epoch: 84 -> Test Accuracy: 82.64
[85, 60] loss: 0.248
[85, 120] loss: 0.244
[85, 180] loss: 0.248
[85, 240] loss: 0.241
[85, 300] loss: 0.252
[85, 360] loss: 0.242
Epoch: 85 -> Loss: 0.178085491061
Epoch: 85 -> Test Accuracy: 82.3
[86, 60] loss: 0.239
[86, 120] loss: 0.258
[86, 180] loss: 0.238
[86, 240] loss: 0.244
[86, 300] loss: 0.235
[86, 360] loss: 0.246
Epoch: 86 -> Loss: 0.247001975775
Epoch: 86 -> Test Accuracy: 82.52
[87, 60] loss: 0.252
[87, 120] loss: 0.246
[87, 180] loss: 0.241
[87, 240] loss: 0.246
[87, 300] loss: 0.249
[87, 360] loss: 0.249
Epoch: 87 -> Loss: 0.407880365849
Epoch: 87 -> Test Accuracy: 82.44
[88, 60] loss: 0.250
[88, 120] loss: 0.242
[88, 180] loss: 0.242
[88, 240] loss: 0.246
[88, 300] loss: 0.240
[88, 360] loss: 0.244
Epoch: 88 -> Loss: 0.29486811161
Epoch: 88 -> Test Accuracy: 82.51
[89, 60

[25, 360] loss: 0.226
Epoch: 25 -> Loss: 0.182897120714
Epoch: 25 -> Test Accuracy: 86.35
[26, 60] loss: 0.204
[26, 120] loss: 0.220
[26, 180] loss: 0.206
[26, 240] loss: 0.238
[26, 300] loss: 0.235
[26, 360] loss: 0.225
Epoch: 26 -> Loss: 0.191764861345
Epoch: 26 -> Test Accuracy: 86.36
[27, 60] loss: 0.212
[27, 120] loss: 0.200
[27, 180] loss: 0.220
[27, 240] loss: 0.213
[27, 300] loss: 0.234
[27, 360] loss: 0.222
Epoch: 27 -> Loss: 0.384246259928
Epoch: 27 -> Test Accuracy: 86.11
[28, 60] loss: 0.202
[28, 120] loss: 0.211
[28, 180] loss: 0.222
[28, 240] loss: 0.216
[28, 300] loss: 0.207
[28, 360] loss: 0.216
Epoch: 28 -> Loss: 0.213461995125
Epoch: 28 -> Test Accuracy: 86.16
[29, 60] loss: 0.197
[29, 120] loss: 0.203
[29, 180] loss: 0.207
[29, 240] loss: 0.209
[29, 300] loss: 0.221
[29, 360] loss: 0.211
Epoch: 29 -> Loss: 0.197112932801
Epoch: 29 -> Test Accuracy: 85.57
[30, 60] loss: 0.199
[30, 120] loss: 0.182
[30, 180] loss: 0.210
[30, 240] loss: 0.209
[30, 300] loss: 0.211
[30, 

Epoch: 66 -> Test Accuracy: 86.26
[67, 60] loss: 0.103
[67, 120] loss: 0.095
[67, 180] loss: 0.096
[67, 240] loss: 0.091
[67, 300] loss: 0.100
[67, 360] loss: 0.096
Epoch: 67 -> Loss: 0.0488171167672
Epoch: 67 -> Test Accuracy: 86.39
[68, 60] loss: 0.096
[68, 120] loss: 0.097
[68, 180] loss: 0.103
[68, 240] loss: 0.098
[68, 300] loss: 0.101
[68, 360] loss: 0.098
Epoch: 68 -> Loss: 0.115463376045
Epoch: 68 -> Test Accuracy: 86.36
[69, 60] loss: 0.093
[69, 120] loss: 0.102
[69, 180] loss: 0.098
[69, 240] loss: 0.096
[69, 300] loss: 0.094
[69, 360] loss: 0.102
Epoch: 69 -> Loss: 0.0813453048468
Epoch: 69 -> Test Accuracy: 86.32
[70, 60] loss: 0.089
[70, 120] loss: 0.089
[70, 180] loss: 0.090
[70, 240] loss: 0.097
[70, 300] loss: 0.094
[70, 360] loss: 0.102
Epoch: 70 -> Loss: 0.0443096607924
Epoch: 70 -> Test Accuracy: 86.38
[71, 60] loss: 0.096
[71, 120] loss: 0.094
[71, 180] loss: 0.090
[71, 240] loss: 0.096
[71, 300] loss: 0.090
[71, 360] loss: 0.093
Epoch: 71 -> Loss: 0.0764009803534
E

...

[32, 180] loss: 0.812
[32, 240] loss: 0.788
[32, 300] loss: 0.808
[32, 360] loss: 0.806
Epoch: 32 -> Loss: 0.81226092577
Epoch: 32 -> Test Accuracy: 65.13
[33, 60] loss: 0.811
[33, 120] loss: 0.803
[33, 180] loss: 0.788
[33, 240] loss: 0.799
[33, 300] loss: 0.815
[33, 360] loss: 0.805
Epoch: 33 -> Loss: 0.799949228764
Epoch: 33 -> Test Accuracy: 65.58
[34, 60] loss: 0.790
[34, 120] loss: 0.816
[34, 180] loss: 0.801
[34, 240] loss: 0.804
[34, 300] loss: 0.789
[34, 360] loss: 0.814
Epoch: 34 -> Loss: 0.731688082218
Epoch: 34 -> Test Accuracy: 65.77
[35, 60] loss: 0.802
[35, 120] loss: 0.801
[35, 180] loss: 0.807
[35, 240] loss: 0.818
[35, 300] loss: 0.800
[35, 360] loss: 0.814
Epoch: 35 -> Loss: 0.876521885395
Epoch: 35 -> Test Accuracy: 65.95
[36, 60] loss: 0.779
[36, 120] loss: 0.815
[36, 180] loss: 0.803
[36, 240] loss: 0.812
[36, 300] loss: 0.807
[36, 360] loss: 0.831
Epoch: 36 -> Loss: 0.79645717144
Epoch: 36 -> Test Accuracy: 65.72
[37, 60] loss: 0.774
[37, 120] loss: 0.784
[37, 18

[73, 360] loss: 0.685
Epoch: 73 -> Loss: 0.884572684765
Epoch: 73 -> Test Accuracy: 68.8
[74, 60] loss: 0.679
[74, 120] loss: 0.667
[74, 180] loss: 0.677
[74, 240] loss: 0.678
[74, 300] loss: 0.673
[74, 360] loss: 0.683
Epoch: 74 -> Loss: 0.975633323193
Epoch: 74 -> Test Accuracy: 68.98
[75, 60] loss: 0.693
[75, 120] loss: 0.692
[75, 180] loss: 0.665
[75, 240] loss: 0.682
[75, 300] loss: 0.674
[75, 360] loss: 0.675
Epoch: 75 -> Loss: 0.729549348354
Epoch: 75 -> Test Accuracy: 69.18
[76, 60] loss: 0.678
[76, 120] loss: 0.663
[76, 180] loss: 0.688
[76, 240] loss: 0.678
[76, 300] loss: 0.682
[76, 360] loss: 0.684
Epoch: 76 -> Loss: 0.819790840149
Epoch: 76 -> Test Accuracy: 68.87
[77, 60] loss: 0.669
[77, 120] loss: 0.659
[77, 180] loss: 0.666
[77, 240] loss: 0.701
[77, 300] loss: 0.664
[77, 360] loss: 0.676
Epoch: 77 -> Loss: 0.771557331085
Epoch: 77 -> Test Accuracy: 68.89
[78, 60] loss: 0.656
[78, 120] loss: 0.689
[78, 180] loss: 0.674
[78, 240] loss: 0.687
[78, 300] loss: 0.661
[78, 3

[15, 120] loss: 1.847
[15, 180] loss: 1.840
[15, 240] loss: 1.851
[15, 300] loss: 1.845
[15, 360] loss: 1.844
Epoch: 15 -> Loss: 1.82569944859
Epoch: 15 -> Test Accuracy: 30.87
[16, 60] loss: 1.836
[16, 120] loss: 1.849
[16, 180] loss: 1.851
[16, 240] loss: 1.836
[16, 300] loss: 1.842
[16, 360] loss: 1.833
Epoch: 16 -> Loss: 1.79934942722
Epoch: 16 -> Test Accuracy: 31.33
[17, 60] loss: 1.833
[17, 120] loss: 1.865
[17, 180] loss: 1.839
[17, 240] loss: 1.828
[17, 300] loss: 1.834
[17, 360] loss: 1.855
Epoch: 17 -> Loss: 1.99478721619
Epoch: 17 -> Test Accuracy: 30.63
[18, 60] loss: 1.843
[18, 120] loss: 1.844
[18, 180] loss: 1.830
[18, 240] loss: 1.836
[18, 300] loss: 1.843
[18, 360] loss: 1.840
Epoch: 18 -> Loss: 2.09351754189
Epoch: 18 -> Test Accuracy: 30.32
[19, 60] loss: 1.849
[19, 120] loss: 1.832
[19, 180] loss: 1.847
[19, 240] loss: 1.842
[19, 300] loss: 1.837
[19, 360] loss: 1.838
Epoch: 19 -> Loss: 1.78403949738
Epoch: 19 -> Test Accuracy: 31.38
[20, 60] loss: 1.852
[20, 120] 

[56, 360] loss: 1.620
Epoch: 56 -> Loss: 1.5257871151
Epoch: 56 -> Test Accuracy: 36.75
[57, 60] loss: 1.607
[57, 120] loss: 1.629
[57, 180] loss: 1.641
[57, 240] loss: 1.615
[57, 300] loss: 1.633
[57, 360] loss: 1.628
Epoch: 57 -> Loss: 1.52280402184
Epoch: 57 -> Test Accuracy: 36.99
[58, 60] loss: 1.623
[58, 120] loss: 1.614
[58, 180] loss: 1.637
[58, 240] loss: 1.604
[58, 300] loss: 1.633
[58, 360] loss: 1.636
Epoch: 58 -> Loss: 1.6641600132
Epoch: 58 -> Test Accuracy: 36.92
[59, 60] loss: 1.625
[59, 120] loss: 1.627
[59, 180] loss: 1.618
[59, 240] loss: 1.613
[59, 300] loss: 1.621
[59, 360] loss: 1.617
Epoch: 59 -> Loss: 1.66690540314
Epoch: 59 -> Test Accuracy: 36.56
[60, 60] loss: 1.622
[60, 120] loss: 1.618
[60, 180] loss: 1.635
[60, 240] loss: 1.594
[60, 300] loss: 1.623
[60, 360] loss: 1.636
Epoch: 60 -> Loss: 1.82768595219
Epoch: 60 -> Test Accuracy: 37.0
[61, 60] loss: 1.631
[61, 120] loss: 1.603
[61, 180] loss: 1.641
[61, 240] loss: 1.622
[61, 300] loss: 1.631
[61, 360] los

[98, 60] loss: 1.598
[98, 120] loss: 1.605
[98, 180] loss: 1.598
[98, 240] loss: 1.599
[98, 300] loss: 1.607
[98, 360] loss: 1.594
Epoch: 98 -> Loss: 1.79247283936
Epoch: 98 -> Test Accuracy: 37.28
[99, 60] loss: 1.604
[99, 120] loss: 1.584
[99, 180] loss: 1.607
[99, 240] loss: 1.632
[99, 300] loss: 1.594
[99, 360] loss: 1.608
Epoch: 99 -> Loss: 1.5799267292
Epoch: 99 -> Test Accuracy: 37.3
[100, 60] loss: 1.604
[100, 120] loss: 1.598
[100, 180] loss: 1.597
[100, 240] loss: 1.588
[100, 300] loss: 1.589
[100, 360] loss: 1.606
Epoch: 100 -> Loss: 1.52403187752
Epoch: 100 -> Test Accuracy: 37.43
Finished Training


In [18]:
# train ConvClassifiers on feature map of net_3block
conv_block5_loss_log, _, conv_block5_test_accuracy_log, _, _ = tr.train_all_blocks(5, 10, [0.1, 0.02, 0.004, 0.0008], 
    [35, 70, 85, 100], 0.9, 5e-4, net_block5, criterion, trainloader, None, testloader, use_ConvClassifier=True) 

[1, 60] loss: 1.373
[1, 120] loss: 1.059
[1, 180] loss: 0.948
[1, 240] loss: 0.896
[1, 300] loss: 0.835
[1, 360] loss: 0.837
Epoch: 1 -> Loss: 0.960916340351
Epoch: 1 -> Test Accuracy: 68.22
[2, 60] loss: 0.763
[2, 120] loss: 0.753
[2, 180] loss: 0.723
[2, 240] loss: 0.691
[2, 300] loss: 0.693
[2, 360] loss: 0.680
Epoch: 2 -> Loss: 0.862775802612
Epoch: 2 -> Test Accuracy: 72.5
[3, 60] loss: 0.628
[3, 120] loss: 0.648
[3, 180] loss: 0.645
[3, 240] loss: 0.635
[3, 300] loss: 0.625
[3, 360] loss: 0.623
Epoch: 3 -> Loss: 0.528761863708
Epoch: 3 -> Test Accuracy: 75.04
[4, 60] loss: 0.602
[4, 120] loss: 0.596
[4, 180] loss: 0.581
[4, 240] loss: 0.588
[4, 300] loss: 0.591
[4, 360] loss: 0.607
Epoch: 4 -> Loss: 0.514687359333
Epoch: 4 -> Test Accuracy: 76.2
[5, 60] loss: 0.587
[5, 120] loss: 0.546
[5, 180] loss: 0.534
[5, 240] loss: 0.562
[5, 300] loss: 0.568
[5, 360] loss: 0.564
Epoch: 5 -> Loss: 0.500259041786
Epoch: 5 -> Test Accuracy: 77.02
[6, 60] loss: 0.535
[6, 120] loss: 0.544
[6, 18

Epoch: 42 -> Loss: 0.198545902967
Epoch: 42 -> Test Accuracy: 85.07
[43, 60] loss: 0.221
[43, 120] loss: 0.233
[43, 180] loss: 0.224
[43, 240] loss: 0.248
[43, 300] loss: 0.241
[43, 360] loss: 0.242
Epoch: 43 -> Loss: 0.23557202518
Epoch: 43 -> Test Accuracy: 85.29
[44, 60] loss: 0.212
[44, 120] loss: 0.229
[44, 180] loss: 0.239
[44, 240] loss: 0.237
[44, 300] loss: 0.251
[44, 360] loss: 0.234
Epoch: 44 -> Loss: 0.309601843357
Epoch: 44 -> Test Accuracy: 84.72
[45, 60] loss: 0.208
[45, 120] loss: 0.228
[45, 180] loss: 0.237
[45, 240] loss: 0.250
[45, 300] loss: 0.248
[45, 360] loss: 0.243
Epoch: 45 -> Loss: 0.24881669879
Epoch: 45 -> Test Accuracy: 84.55
[46, 60] loss: 0.228
[46, 120] loss: 0.224
[46, 180] loss: 0.237
[46, 240] loss: 0.221
[46, 300] loss: 0.236
[46, 360] loss: 0.241
Epoch: 46 -> Loss: 0.200550347567
Epoch: 46 -> Test Accuracy: 84.48
[47, 60] loss: 0.220
[47, 120] loss: 0.223
[47, 180] loss: 0.239
[47, 240] loss: 0.214
[47, 300] loss: 0.234
[47, 360] loss: 0.238
Epoch: 

[84, 60] loss: 0.103
[84, 120] loss: 0.106
[84, 180] loss: 0.112
[84, 240] loss: 0.109
[84, 300] loss: 0.107
[84, 360] loss: 0.112
Epoch: 84 -> Loss: 0.174443304539
Epoch: 84 -> Test Accuracy: 85.95
[85, 60] loss: 0.101
[85, 120] loss: 0.097
[85, 180] loss: 0.106
[85, 240] loss: 0.108
[85, 300] loss: 0.114
[85, 360] loss: 0.114
Epoch: 85 -> Loss: 0.122100234032
Epoch: 85 -> Test Accuracy: 86.1
[86, 60] loss: 0.096
[86, 120] loss: 0.099
[86, 180] loss: 0.094
[86, 240] loss: 0.098
[86, 300] loss: 0.095
[86, 360] loss: 0.092
Epoch: 86 -> Loss: 0.0955424308777
Epoch: 86 -> Test Accuracy: 86.22
[87, 60] loss: 0.091
[87, 120] loss: 0.095
[87, 180] loss: 0.087
[87, 240] loss: 0.090
[87, 300] loss: 0.092
[87, 360] loss: 0.090
Epoch: 87 -> Loss: 0.0658046081662
Epoch: 87 -> Test Accuracy: 85.98
[88, 60] loss: 0.093
[88, 120] loss: 0.089
[88, 180] loss: 0.092
[88, 240] loss: 0.095
[88, 300] loss: 0.088
[88, 360] loss: 0.093
Epoch: 88 -> Loss: 0.112874865532
Epoch: 88 -> Test Accuracy: 86.13
[89,

[25, 300] loss: 0.267
[25, 360] loss: 0.289
Epoch: 25 -> Loss: 0.23557433486
Epoch: 25 -> Test Accuracy: 84.91
[26, 60] loss: 0.260
[26, 120] loss: 0.243
[26, 180] loss: 0.269
[26, 240] loss: 0.267
[26, 300] loss: 0.268
[26, 360] loss: 0.279
Epoch: 26 -> Loss: 0.32897824049
Epoch: 26 -> Test Accuracy: 85.77
[27, 60] loss: 0.231
[27, 120] loss: 0.246
[27, 180] loss: 0.262
[27, 240] loss: 0.279
[27, 300] loss: 0.271
[27, 360] loss: 0.278
Epoch: 27 -> Loss: 0.184163317084
Epoch: 27 -> Test Accuracy: 85.5
[28, 60] loss: 0.244
[28, 120] loss: 0.248
[28, 180] loss: 0.257
[28, 240] loss: 0.272
[28, 300] loss: 0.277
[28, 360] loss: 0.281
Epoch: 28 -> Loss: 0.184274703264
Epoch: 28 -> Test Accuracy: 85.56
[29, 60] loss: 0.234
[29, 120] loss: 0.241
[29, 180] loss: 0.253
[29, 240] loss: 0.264
[29, 300] loss: 0.271
[29, 360] loss: 0.284
Epoch: 29 -> Loss: 0.172003015876
Epoch: 29 -> Test Accuracy: 85.08
[30, 60] loss: 0.237
[30, 120] loss: 0.248
[30, 180] loss: 0.270
[30, 240] loss: 0.278
[30, 300

Epoch: 66 -> Loss: 0.110937856138
Epoch: 66 -> Test Accuracy: 86.83
[67, 60] loss: 0.096
[67, 120] loss: 0.093
[67, 180] loss: 0.090
[67, 240] loss: 0.102
[67, 300] loss: 0.103
[67, 360] loss: 0.114
Epoch: 67 -> Loss: 0.0884017944336
Epoch: 67 -> Test Accuracy: 87.02
[68, 60] loss: 0.086
[68, 120] loss: 0.087
[68, 180] loss: 0.097
[68, 240] loss: 0.101
[68, 300] loss: 0.107
[68, 360] loss: 0.112
Epoch: 68 -> Loss: 0.0737691298127
Epoch: 68 -> Test Accuracy: 87.4
[69, 60] loss: 0.097
[69, 120] loss: 0.093
[69, 180] loss: 0.105
[69, 240] loss: 0.112
[69, 300] loss: 0.118
[69, 360] loss: 0.111
Epoch: 69 -> Loss: 0.145862191916
Epoch: 69 -> Test Accuracy: 86.93
[70, 60] loss: 0.088
[70, 120] loss: 0.087
[70, 180] loss: 0.097
[70, 240] loss: 0.107
[70, 300] loss: 0.103
[70, 360] loss: 0.110
Epoch: 70 -> Loss: 0.100049495697
Epoch: 70 -> Test Accuracy: 86.98
[71, 60] loss: 0.084
[71, 120] loss: 0.065
[71, 180] loss: 0.059
[71, 240] loss: 0.064
[71, 300] loss: 0.057
[71, 360] loss: 0.059
Epoc

...

[32, 120] loss: 0.845
[32, 180] loss: 0.833
[32, 240] loss: 0.840
[32, 300] loss: 0.838
[32, 360] loss: 0.856
Epoch: 32 -> Loss: 0.867956757545
Epoch: 32 -> Test Accuracy: 64.03
[33, 60] loss: 0.844
[33, 120] loss: 0.818
[33, 180] loss: 0.856
[33, 240] loss: 0.823
[33, 300] loss: 0.820
[33, 360] loss: 0.848
Epoch: 33 -> Loss: 0.693422138691
Epoch: 33 -> Test Accuracy: 64.79
[34, 60] loss: 0.839
[34, 120] loss: 0.846
[34, 180] loss: 0.824
[34, 240] loss: 0.835
[34, 300] loss: 0.849
[34, 360] loss: 0.818
Epoch: 34 -> Loss: 0.751186430454
Epoch: 34 -> Test Accuracy: 64.56
[35, 60] loss: 0.836
[35, 120] loss: 0.831
[35, 180] loss: 0.826
[35, 240] loss: 0.836
[35, 300] loss: 0.839
[35, 360] loss: 0.855
Epoch: 35 -> Loss: 1.16005349159
Epoch: 35 -> Test Accuracy: 64.56
[36, 60] loss: 0.771
[36, 120] loss: 0.740
[36, 180] loss: 0.759
[36, 240] loss: 0.737
[36, 300] loss: 0.731
[36, 360] loss: 0.754
Epoch: 36 -> Loss: 0.740938663483
Epoch: 36 -> Test Accuracy: 68.36
[37, 60] loss: 0.739
[37, 1

[73, 240] loss: 0.633
[73, 300] loss: 0.631
[73, 360] loss: 0.611
Epoch: 73 -> Loss: 0.559405624866
Epoch: 73 -> Test Accuracy: 70.45
[74, 60] loss: 0.626
[74, 120] loss: 0.612
[74, 180] loss: 0.601
[74, 240] loss: 0.608
[74, 300] loss: 0.622
[74, 360] loss: 0.619
Epoch: 74 -> Loss: 0.806261837482
Epoch: 74 -> Test Accuracy: 71.1
[75, 60] loss: 0.600
[75, 120] loss: 0.605
[75, 180] loss: 0.628
[75, 240] loss: 0.624
[75, 300] loss: 0.600
[75, 360] loss: 0.616
Epoch: 75 -> Loss: 0.652823030949
Epoch: 75 -> Test Accuracy: 70.89
[76, 60] loss: 0.615
[76, 120] loss: 0.622
[76, 180] loss: 0.627
[76, 240] loss: 0.606
[76, 300] loss: 0.628
[76, 360] loss: 0.600
Epoch: 76 -> Loss: 0.598550617695
Epoch: 76 -> Test Accuracy: 71.2
[77, 60] loss: 0.602
[77, 120] loss: 0.603
[77, 180] loss: 0.632
[77, 240] loss: 0.611
[77, 300] loss: 0.612
[77, 360] loss: 0.611
Epoch: 77 -> Loss: 0.556010365486
Epoch: 77 -> Test Accuracy: 71.33
[78, 60] loss: 0.611
[78, 120] loss: 0.614
[78, 180] loss: 0.631
[78, 24

[15, 60] loss: 1.773
[15, 120] loss: 1.762
[15, 180] loss: 1.766
[15, 240] loss: 1.784
[15, 300] loss: 1.761
[15, 360] loss: 1.757
Epoch: 15 -> Loss: 1.85305559635
Epoch: 15 -> Test Accuracy: 32.46
[16, 60] loss: 1.770
[16, 120] loss: 1.756
[16, 180] loss: 1.775
[16, 240] loss: 1.751
[16, 300] loss: 1.769
[16, 360] loss: 1.764
Epoch: 16 -> Loss: 1.73094022274
Epoch: 16 -> Test Accuracy: 33.73
[17, 60] loss: 1.774
[17, 120] loss: 1.762
[17, 180] loss: 1.769
[17, 240] loss: 1.743
[17, 300] loss: 1.777
[17, 360] loss: 1.749
Epoch: 17 -> Loss: 1.65575253963
Epoch: 17 -> Test Accuracy: 33.26
[18, 60] loss: 1.765
[18, 120] loss: 1.749
[18, 180] loss: 1.760
[18, 240] loss: 1.765
[18, 300] loss: 1.769
[18, 360] loss: 1.764
Epoch: 18 -> Loss: 1.99318289757
Epoch: 18 -> Test Accuracy: 33.83
[19, 60] loss: 1.750
[19, 120] loss: 1.753
[19, 180] loss: 1.781
[19, 240] loss: 1.760
[19, 300] loss: 1.766
[19, 360] loss: 1.748
Epoch: 19 -> Loss: 1.71245706081
Epoch: 19 -> Test Accuracy: 33.51
[20, 60] l

[56, 300] loss: 1.617
[56, 360] loss: 1.638
Epoch: 56 -> Loss: 1.51970827579
Epoch: 56 -> Test Accuracy: 38.12
[57, 60] loss: 1.623
[57, 120] loss: 1.644
[57, 180] loss: 1.633
[57, 240] loss: 1.637
[57, 300] loss: 1.604
[57, 360] loss: 1.633
Epoch: 57 -> Loss: 1.48829591274
Epoch: 57 -> Test Accuracy: 38.61
[58, 60] loss: 1.611
[58, 120] loss: 1.633
[58, 180] loss: 1.635
[58, 240] loss: 1.623
[58, 300] loss: 1.636
[58, 360] loss: 1.610
Epoch: 58 -> Loss: 1.69478416443
Epoch: 58 -> Test Accuracy: 39.03
[59, 60] loss: 1.610
[59, 120] loss: 1.620
[59, 180] loss: 1.621
[59, 240] loss: 1.623
[59, 300] loss: 1.612
[59, 360] loss: 1.636
Epoch: 59 -> Loss: 1.80027079582
Epoch: 59 -> Test Accuracy: 39.03
[60, 60] loss: 1.646
[60, 120] loss: 1.623
[60, 180] loss: 1.617
[60, 240] loss: 1.627
[60, 300] loss: 1.626
[60, 360] loss: 1.630
Epoch: 60 -> Loss: 1.6705083847
Epoch: 60 -> Test Accuracy: 37.53
[61, 60] loss: 1.629
[61, 120] loss: 1.631
[61, 180] loss: 1.627
[61, 240] loss: 1.627
[61, 300] l

[98, 60] loss: 1.474
[98, 120] loss: 1.485
[98, 180] loss: 1.493
[98, 240] loss: 1.481
[98, 300] loss: 1.515
[98, 360] loss: 1.481
Epoch: 98 -> Loss: 1.41757082939
Epoch: 98 -> Test Accuracy: 41.27
[99, 60] loss: 1.490
[99, 120] loss: 1.480
[99, 180] loss: 1.494
[99, 240] loss: 1.503
[99, 300] loss: 1.484
[99, 360] loss: 1.499
Epoch: 99 -> Loss: 1.64877724648
Epoch: 99 -> Test Accuracy: 41.31
[100, 60] loss: 1.507
[100, 120] loss: 1.494
[100, 180] loss: 1.490
[100, 240] loss: 1.487
[100, 300] loss: 1.486
[100, 360] loss: 1.488
Epoch: 100 -> Loss: 1.58387207985
Epoch: 100 -> Test Accuracy: 41.62
Finished Training


In [ ]:
# save variables
fm.save_variable([rot_block5_loss_log, rot_block5_test_accuracy_log, 
                  block5_loss_log, block5_test_accuracy_log, 
                  conv_block5_loss_log, conv_block5_test_accuracy_log], "5_block_net")

In [19]:
# rename files
fm.add_block_to_name(5, [100, 200])

# Supervised NIN

Note: In the code of the paper a 3 convolutional block RotNet was used for the classification task.

In [20]:
# initialize networks
net_class = RN.RotNet(num_classes=10, num_conv_block=3, add_avg_pool=False)

In [21]:
# train 3 block RotNet on classification task
class_NIN_loss_log, _, class_NIN_test_accuracy_log, _, _ = tr.adaptive_learning([0.1, 0.02, 0.004, 0.0008], 
    [60, 120, 160, 200], 0.9, 5e-4, net_class, criterion, trainloader, None, testloader)

[1, 60] loss: 1.819
[1, 120] loss: 1.489
[1, 180] loss: 1.363
[1, 240] loss: 1.256
[1, 300] loss: 1.179
[1, 360] loss: 1.129
Epoch: 1 -> Loss: 1.23693716526
Epoch: 1 -> Test Accuracy: 58.58
[2, 60] loss: 1.034
[2, 120] loss: 0.983
[2, 180] loss: 0.963
[2, 240] loss: 0.905
[2, 300] loss: 0.882
[2, 360] loss: 0.872
Epoch: 2 -> Loss: 0.986335098743
Epoch: 2 -> Test Accuracy: 66.94
[3, 60] loss: 0.809
[3, 120] loss: 0.828
[3, 180] loss: 0.776
[3, 240] loss: 0.776
[3, 300] loss: 0.753
[3, 360] loss: 0.761
Epoch: 3 -> Loss: 0.888205885887
Epoch: 3 -> Test Accuracy: 72.76
[4, 60] loss: 0.732
[4, 120] loss: 0.715
[4, 180] loss: 0.724
[4, 240] loss: 0.719
[4, 300] loss: 0.702
[4, 360] loss: 0.691
Epoch: 4 -> Loss: 0.861030876637
Epoch: 4 -> Test Accuracy: 73.76
[5, 60] loss: 0.649
[5, 120] loss: 0.644
[5, 180] loss: 0.679
[5, 240] loss: 0.675
[5, 300] loss: 0.669
[5, 360] loss: 0.657
Epoch: 5 -> Loss: 0.828762710094
Epoch: 5 -> Test Accuracy: 76.76
[6, 60] loss: 0.627
[6, 120] loss: 0.621
[6, 1

Epoch: 42 -> Loss: 0.497273266315
Epoch: 42 -> Test Accuracy: 82.86
[43, 60] loss: 0.383
[43, 120] loss: 0.398
[43, 180] loss: 0.406
[43, 240] loss: 0.396
[43, 300] loss: 0.392
[43, 360] loss: 0.413
Epoch: 43 -> Loss: 0.391762316227
Epoch: 43 -> Test Accuracy: 83.12
[44, 60] loss: 0.356
[44, 120] loss: 0.401
[44, 180] loss: 0.408
[44, 240] loss: 0.405
[44, 300] loss: 0.417
[44, 360] loss: 0.393
Epoch: 44 -> Loss: 0.49181586504
Epoch: 44 -> Test Accuracy: 83.01
[45, 60] loss: 0.386
[45, 120] loss: 0.394
[45, 180] loss: 0.376
[45, 240] loss: 0.426
[45, 300] loss: 0.380
[45, 360] loss: 0.422
Epoch: 45 -> Loss: 0.550320327282
Epoch: 45 -> Test Accuracy: 83.18
[46, 60] loss: 0.376
[46, 120] loss: 0.386
[46, 180] loss: 0.388
[46, 240] loss: 0.410
[46, 300] loss: 0.383
[46, 360] loss: 0.407
Epoch: 46 -> Loss: 0.319671213627
Epoch: 46 -> Test Accuracy: 82.69
[47, 60] loss: 0.375
[47, 120] loss: 0.393
[47, 180] loss: 0.391
[47, 240] loss: 0.387
[47, 300] loss: 0.382
[47, 360] loss: 0.406
Epoch:

[84, 60] loss: 0.110
[84, 120] loss: 0.129
[84, 180] loss: 0.154
[84, 240] loss: 0.145
[84, 300] loss: 0.134
[84, 360] loss: 0.151
Epoch: 84 -> Loss: 0.112958587706
Epoch: 84 -> Test Accuracy: 88.0
[85, 60] loss: 0.124
[85, 120] loss: 0.123
[85, 180] loss: 0.136
[85, 240] loss: 0.139
[85, 300] loss: 0.148
[85, 360] loss: 0.132
Epoch: 85 -> Loss: 0.230785563588
Epoch: 85 -> Test Accuracy: 88.94
[86, 60] loss: 0.129
[86, 120] loss: 0.125
[86, 180] loss: 0.135
[86, 240] loss: 0.141
[86, 300] loss: 0.143
[86, 360] loss: 0.136
Epoch: 86 -> Loss: 0.0781771987677
Epoch: 86 -> Test Accuracy: 88.91
[87, 60] loss: 0.121
[87, 120] loss: 0.124
[87, 180] loss: 0.125
[87, 240] loss: 0.126
[87, 300] loss: 0.133
[87, 360] loss: 0.141
Epoch: 87 -> Loss: 0.148603647947
Epoch: 87 -> Test Accuracy: 88.37
[88, 60] loss: 0.132
[88, 120] loss: 0.125
[88, 180] loss: 0.145
[88, 240] loss: 0.136
[88, 300] loss: 0.132
[88, 360] loss: 0.149
Epoch: 88 -> Loss: 0.146424606442
Epoch: 88 -> Test Accuracy: 88.25
[89, 

[124, 180] loss: 0.023
[124, 240] loss: 0.024
[124, 300] loss: 0.022
[124, 360] loss: 0.024
Epoch: 124 -> Loss: 0.0201849993318
Epoch: 124 -> Test Accuracy: 91.13
[125, 60] loss: 0.019
[125, 120] loss: 0.019
[125, 180] loss: 0.020
[125, 240] loss: 0.017
[125, 300] loss: 0.021
[125, 360] loss: 0.019
Epoch: 125 -> Loss: 0.0156586822122
Epoch: 125 -> Test Accuracy: 91.49
[126, 60] loss: 0.019
[126, 120] loss: 0.016
[126, 180] loss: 0.016
[126, 240] loss: 0.015
[126, 300] loss: 0.017
[126, 360] loss: 0.018
Epoch: 126 -> Loss: 0.0122456075624
Epoch: 126 -> Test Accuracy: 91.34
[127, 60] loss: 0.015
[127, 120] loss: 0.017
[127, 180] loss: 0.017
[127, 240] loss: 0.015
[127, 300] loss: 0.016
[127, 360] loss: 0.017
Epoch: 127 -> Loss: 0.0120175955817
Epoch: 127 -> Test Accuracy: 91.11
[128, 60] loss: 0.016
[128, 120] loss: 0.016
[128, 180] loss: 0.016
[128, 240] loss: 0.015
[128, 300] loss: 0.015
[128, 360] loss: 0.015
Epoch: 128 -> Loss: 0.0307856090367
Epoch: 128 -> Test Accuracy: 91.33
[129,

Epoch: 163 -> Loss: 0.00582162756473
Epoch: 163 -> Test Accuracy: 91.28
[164, 60] loss: 0.007
[164, 120] loss: 0.007
[164, 180] loss: 0.007
[164, 240] loss: 0.006
[164, 300] loss: 0.007
[164, 360] loss: 0.007
Epoch: 164 -> Loss: 0.00586801767349
Epoch: 164 -> Test Accuracy: 91.46
[165, 60] loss: 0.006
[165, 120] loss: 0.007
[165, 180] loss: 0.007
[165, 240] loss: 0.007
[165, 300] loss: 0.007
[165, 360] loss: 0.007
Epoch: 165 -> Loss: 0.0117396656424
Epoch: 165 -> Test Accuracy: 91.41
[166, 60] loss: 0.006
[166, 120] loss: 0.006
[166, 180] loss: 0.007
[166, 240] loss: 0.006
[166, 300] loss: 0.006
[166, 360] loss: 0.007
Epoch: 166 -> Loss: 0.00971355475485
Epoch: 166 -> Test Accuracy: 91.47
[167, 60] loss: 0.006
[167, 120] loss: 0.007
[167, 180] loss: 0.006
[167, 240] loss: 0.006
[167, 300] loss: 0.007
[167, 360] loss: 0.007
Epoch: 167 -> Loss: 0.0403599776328
Epoch: 167 -> Test Accuracy: 91.45
[168, 60] loss: 0.006
[168, 120] loss: 0.008
[168, 180] loss: 0.007
[168, 240] loss: 0.006
[16

In [ ]:
# save variables
fm.save_variable([class_NIN_loss_log, class_NIN_test_accuracy_log], "supervised_NIN")

# Semi-supervised Learning

In [6]:
# initialize networks
semi_net = fm.load_net("RotNet_rotation_200_4_block_net")

In [7]:
semi_loss_log, semi_accuracy_log, super_loss_log, super_accuracy_log = tr.train_semi([20, 100, 400, 1000, 5000], 10, 
    trainset, testset, 128, [0.1, 0.02, 0.004, 0.0008], [35, 70, 85, 100], [0.1, 0.02, 0.004, 0.0008],
               [60, 120, 160, 200], 0.9, 5e-4, semi_net, criterion)

Epoch: 1 -> Loss: 2.5183327198
Epoch: 1 -> Test Accuracy: 34.39
Epoch: 2 -> Loss: 1.3776679039
Epoch: 2 -> Test Accuracy: 48.19
Epoch: 3 -> Loss: 0.986231267452
Epoch: 3 -> Test Accuracy: 54.01
Epoch: 4 -> Loss: 0.719306468964
Epoch: 4 -> Test Accuracy: 57.46
Epoch: 5 -> Loss: 0.477100372314
Epoch: 5 -> Test Accuracy: 58.43
Epoch: 6 -> Loss: 0.42391577363
Epoch: 6 -> Test Accuracy: 59.69
Epoch: 7 -> Loss: 0.285415709019
Epoch: 7 -> Test Accuracy: 60.01
Epoch: 8 -> Loss: 0.238389223814
Epoch: 8 -> Test Accuracy: 60.15
Epoch: 9 -> Loss: 0.202291429043
Epoch: 9 -> Test Accuracy: 60.49
Epoch: 10 -> Loss: 0.115706339478
Epoch: 10 -> Test Accuracy: 61.01
Epoch: 11 -> Loss: 0.106966279447
Epoch: 11 -> Test Accuracy: 60.95
Epoch: 12 -> Loss: 0.100582882762
Epoch: 12 -> Test Accuracy: 60.91
Epoch: 13 -> Loss: 0.0855379924178
Epoch: 13 -> Test Accuracy: 61.07
Epoch: 14 -> Loss: 0.0876723229885
Epoch: 14 -> Test Accuracy: 61.07
Epoch: 15 -> Loss: 0.0521995685995
Epoch: 15 -> Test Accuracy: 60.88


TypeError: argument of type 'bool' is not iterable

In [ ]:
# save variables
fm.save_variable([semi_loss_log, semi_accuracy_log, super_loss_log, super_accuracy_log], "semi-supervised")

# Evaluate Test Accuracies

In [10]:
# 3 ConvBlock RotNet model and Classifiers
ev.evaluate_all(3, testloader, classes)

Evaluating RotNet model with 3 Convolutional Blocks:
()
Evaluating Rotation Task:
Test Accuracy:  92.180 %
Test Accuracy of original :  92.190 %
Test Accuracy of 90 rotation :  92.360 %
Test Accuracy of 180 rotation :  92.010 %
Test Accuracy of 270 rotation :  92.160 %
()
--------------------------------------------------------------------------------
()
Starting to evaluate Non-Linear Classifier:
()
Evaluating Non-Linear Classifier on Convolutional Block 1:
Test Accuracy:  83.110 %
Test Accuracy of plane :  85.100 %
Test Accuracy of car :  90.400 %
Test Accuracy of bird :  74.800 %
Test Accuracy of cat :  69.400 %
Test Accuracy of deer :  80.200 %
Test Accuracy of dog :  73.700 %
Test Accuracy of frog :  88.200 %
Test Accuracy of horse :  87.900 %
Test Accuracy of ship :  90.500 %
Test Accuracy of truck :  90.900 %
()
Evaluating Non-Linear Classifier on Convolutional Block 2:
Test Accuracy:  86.540 %
Test Accuracy of plane :  88.600 %
Test Accuracy of car :  92.500 %
Test Accuracy of 

{'Accuracy ConvClassifier ConvBlock 1': 86.4,
 'Accuracy ConvClassifier ConvBlock 2': 88.71,
 'Accuracy ConvClassifier ConvBlock 3': 62.75,
 'Accuracy Non-Linear ConvBlock 1': 83.11,
 'Accuracy Non-Linear ConvBlock 2': 86.54,
 'Accuracy Non-Linear ConvBlock 3': 53.6,
 'Accuracy Rotation Task': 92.18,
 'Class Accuracy ConvClassifier ConvBlock 1': [87.3,
  93.7,
  80.0,
  77.5,
  84.0,
  80.0,
  89.6,
  88.7,
  90.8,
  92.4],
 'Class Accuracy ConvClassifier ConvBlock 2': [89.2,
  93.9,
  83.5,
  80.5,
  90.6,
  82.7,
  93.1,
  90.2,
  92.7,
  90.7],
 'Class Accuracy ConvClassifier ConvBlock 3': [68.4,
  65.4,
  47.0,
  49.9,
  58.3,
  57.2,
  74.3,
  65.5,
  70.5,
  71.0],
 'Class Accuracy Non-Linear ConvBlock 1': [85.1,
  90.4,
  74.8,
  69.4,
  80.2,
  73.7,
  88.2,
  87.9,
  90.5,
  90.9],
 'Class Accuracy Non-Linear ConvBlock 2': [88.6,
  92.5,
  81.6,
  77.1,
  85.5,
  78.8,
  89.7,
  88.3,
  92.8,
  90.5],
 'Class Accuracy Non-Linear ConvBlock 3': [63.0,
  56.1,
  42.2,
  38.7,
  4

In [11]:
# 4 ConvBlock RotNet model and Classifiers
ev.evaluate_all(4, testloader, classes)

Evaluating RotNet model with 4 Convolutional Blocks:
()
Evaluating Rotation Task:
Test Accuracy:  92.573 %
Test Accuracy of original :  93.040 %
Test Accuracy of 90 rotation :  92.380 %
Test Accuracy of 180 rotation :  92.310 %
Test Accuracy of 270 rotation :  92.560 %
()
--------------------------------------------------------------------------------
()
Starting to evaluate Non-Linear Classifier:
()
Evaluating Non-Linear Classifier on Convolutional Block 1:
Test Accuracy:  83.140 %
Test Accuracy of plane :  85.000 %
Test Accuracy of car :  91.600 %
Test Accuracy of bird :  75.400 %
Test Accuracy of cat :  68.100 %
Test Accuracy of deer :  80.400 %
Test Accuracy of dog :  73.800 %
Test Accuracy of frog :  88.900 %
Test Accuracy of horse :  86.900 %
Test Accuracy of ship :  91.400 %
Test Accuracy of truck :  89.900 %
()
Evaluating Non-Linear Classifier on Convolutional Block 2:
Test Accuracy:  87.050 %
Test Accuracy of plane :  89.100 %
Test Accuracy of car :  92.600 %
Test Accuracy of 

{'Accuracy ConvClassifier ConvBlock 1': 86.59,
 'Accuracy ConvClassifier ConvBlock 2': 88.77,
 'Accuracy ConvClassifier ConvBlock 3': 84.46,
 'Accuracy ConvClassifier ConvBlock 4': 52.84,
 'Accuracy Non-Linear ConvBlock 1': 83.14,
 'Accuracy Non-Linear ConvBlock 2': 87.05,
 'Accuracy Non-Linear ConvBlock 3': 82.06,
 'Accuracy Non-Linear ConvBlock 4': 45.45,
 'Accuracy Rotation Task': 92.5725,
 'Class Accuracy ConvClassifier ConvBlock 1': [89.5,
  92.5,
  81.0,
  74.8,
  84.2,
  79.4,
  91.6,
  89.4,
  91.7,
  91.8],
 'Class Accuracy ConvClassifier ConvBlock 2': [90.3,
  92.9,
  84.7,
  80.9,
  89.7,
  81.0,
  92.1,
  91.3,
  92.1,
  92.7],
 'Class Accuracy ConvClassifier ConvBlock 3': [86.7,
  89.0,
  79.6,
  75.3,
  85.7,
  75.3,
  87.2,
  88.3,
  88.0,
  89.5],
 'Class Accuracy ConvClassifier ConvBlock 4': [59.3,
  57.6,
  46.2,
  41.1,
  43.4,
  45.9,
  59.3,
  56.5,
  62.7,
  56.4],
 'Class Accuracy Non-Linear ConvBlock 1': [85.0,
  91.6,
  75.4,
  68.1,
  80.4,
  73.8,
  88.9,
  8

In [12]:
# 5 ConvBlock RotNet model and Classifiers
ev.evaluate_all(5, testloader, classes)

Evaluating RotNet model with 5 Convolutional Blocks:
()
Evaluating Rotation Task:
Test Accuracy:  91.830 %
Test Accuracy of original :  91.790 %
Test Accuracy of 90 rotation :  91.900 %
Test Accuracy of 180 rotation :  92.470 %
Test Accuracy of 270 rotation :  91.160 %
()
--------------------------------------------------------------------------------
()
Starting to evaluate Non-Linear Classifier:
()
Evaluating Non-Linear Classifier on Convolutional Block 1:
Test Accuracy:  82.380 %
Test Accuracy of plane :  83.100 %
Test Accuracy of car :  88.700 %
Test Accuracy of bird :  76.900 %
Test Accuracy of cat :  68.700 %
Test Accuracy of deer :  80.400 %
Test Accuracy of dog :  72.600 %
Test Accuracy of frog :  88.200 %
Test Accuracy of horse :  85.500 %
Test Accuracy of ship :  92.000 %
Test Accuracy of truck :  87.700 %
()
Evaluating Non-Linear Classifier on Convolutional Block 2:
Test Accuracy:  86.040 %
Test Accuracy of plane :  87.600 %
Test Accuracy of car :  92.600 %
Test Accuracy of 

{'Accuracy ConvClassifier ConvBlock 1': 85.93,
 'Accuracy ConvClassifier ConvBlock 2': 88.62,
 'Accuracy ConvClassifier ConvBlock 3': 84.59,
 'Accuracy ConvClassifier ConvBlock 4': 71.66,
 'Accuracy ConvClassifier ConvBlock 5': 41.62,
 'Accuracy Non-Linear ConvBlock 1': 82.38,
 'Accuracy Non-Linear ConvBlock 2': 86.04,
 'Accuracy Non-Linear ConvBlock 3': 82.61,
 'Accuracy Non-Linear ConvBlock 4': 69.25,
 'Accuracy Non-Linear ConvBlock 5': 37.43,
 'Accuracy Rotation Task': 91.83,
 'Class Accuracy ConvClassifier ConvBlock 1': [88.6,
  92.2,
  80.3,
  75.3,
  83.7,
  78.9,
  90.8,
  88.3,
  91.1,
  90.1],
 'Class Accuracy ConvClassifier ConvBlock 2': [88.9,
  93.9,
  84.8,
  78.8,
  89.4,
  83.4,
  91.7,
  90.8,
  92.1,
  92.4],
 'Class Accuracy ConvClassifier ConvBlock 3': [86.4,
  89.7,
  80.0,
  72.2,
  84.9,
  77.6,
  89.4,
  85.7,
  89.2,
  90.8],
 'Class Accuracy ConvClassifier ConvBlock 4': [76.8,
  78.6,
  62.7,
  57.7,
  69.8,
  60.6,
  82.5,
  76.8,
  75.2,
  75.9],
 'Class Accu

In [13]:
# Supervised NIN
ev.evaluate_all(0, testloader, classes)

Evaluating Supervised NIN Classification Task:
Test Accuracy:  91.550 %
Test Accuracy of plane :  91.900 %
Test Accuracy of car :  96.000 %
Test Accuracy of bird :  88.000 %
Test Accuracy of cat :  82.100 %
Test Accuracy of deer :  92.400 %
Test Accuracy of dog :  86.000 %
Test Accuracy of frog :  95.100 %
Test Accuracy of horse :  93.200 %
Test Accuracy of ship :  96.200 %
Test Accuracy of truck :  94.600 %


{'Accuracy Supervised NIN': 91.55,
 'Class Accuracy Supervised NIN': [91.9,
  96.0,
  88.0,
  82.1,
  92.4,
  86.0,
  95.1,
  93.2,
  96.2,
  94.6]}